In [1]:
from __future__ import annotations

import re
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, Union, Callable

from rdflib import Graph, Literal, Namespace, URIRef

AG = Namespace("http://www.semanticweb.org/AgentProgramParams/")


# -----------------------------
# Tracer
# -----------------------------

@dataclass
class Tracer:
    enabled: bool = True
    print_live: bool = True
    lines: List[str] = None

    def __post_init__(self):
        if self.lines is None:
            self.lines = []

    def log(self, msg: str) -> None:
        if not self.enabled:
            return
        self.lines.append(msg)
        if self.print_live:
            print(msg)


# -----------------------------
# 1) FBD Python Export Analyse
# -----------------------------

@dataclass
class FbdNode:
    var: str
    func: Optional[str] = None
    args: Optional[List["FbdNode"]] = None

    def to_dict(self) -> Dict[str, Any]:
        if not self.func:
            return {"var": self.var}
        return {"var": self.var, "func": self.func, "args": [a.to_dict() for a in (self.args or [])]}


def _parse_fbd_call_assignments(fbd_py_code: str, trace: Optional[Tracer] = None) -> Dict[str, Tuple[str, List[str]]]:
    pattern = r"^\s*(V_\d+)\s*=\s*([A-Za-z_][A-Za-z0-9_]*)\(([^)]*)\)\s*$"
    m: Dict[str, Tuple[str, List[str]]] = {}
    for mm in re.finditer(pattern, fbd_py_code, flags=re.M):
        var, func, argstr = mm.group(1), mm.group(2), mm.group(3)
        args = [a.strip() for a in argstr.split(",") if a.strip()]
        m[var] = (func, args)
    if trace:
        trace.log(f"[FBD] Parsed call assignments: {len(m)} entries (V_* = FUNC(args...))")
    return m


def _find_final_state_assignment_var(fbd_py_code: str, state_name: str = "D2", trace: Optional[Tracer] = None) -> Optional[str]:
    m = re.search(rf"^\s*{re.escape(state_name)}\s*=\s*(V_\d+)\s*$", fbd_py_code, flags=re.M)
    if trace:
        if m:
            trace.log(f"[FBD] Found final state assignment: {state_name} = {m.group(1)}")
        else:
            trace.log(f"[FBD] Did not find final state assignment for {state_name}")
    return m.group(1) if m else None


def _build_fbd_tree(
    var: str,
    assigns: Dict[str, Tuple[str, List[str]]],
    depth: int = 0,
    max_depth: int = 25,
) -> FbdNode:
    if depth >= max_depth:
        return FbdNode(var=var, func="MAX_DEPTH", args=[])

    if var not in assigns:
        return FbdNode(var=var)

    func, args = assigns[var]
    return FbdNode(var=var, func=func, args=[_build_fbd_tree(a, assigns, depth + 1, max_depth) for a in args])


def trace_d2_set_reset_from_fbd(fbd_py_code: str, state_name: str = "D2", trace: Optional[Tracer] = None) -> Dict[str, Any]:
    assigns = _parse_fbd_call_assignments(fbd_py_code, trace=trace)
    d2_from = _find_final_state_assignment_var(fbd_py_code, state_name=state_name, trace=trace)
    if not d2_from:
        return {"error": f"Konnte '{state_name} = V_...' im Code nicht finden."}

    if d2_from not in assigns:
        return {"error": f"'{state_name}' kommt von '{d2_from}', aber '{d2_from}' hat keine Call-Zuweisung im Code."}

    func, args = assigns[d2_from]
    if trace:
        trace.log(f"[FBD] {d2_from} is computed by {func} with args {args}")

    if not (func.startswith("RS_") and len(args) >= 2):
        full_tree = _build_fbd_tree(d2_from, assigns).to_dict()
        if trace:
            trace.log("[FBD] Assignment is not RS_* (set reset). Returning generic dependency tree.")
        return {"state": state_name, "assigned_from": d2_from, "tree": full_tree}

    set_var, reset_var = args[0], args[1]
    if trace:
        trace.log(f"[FBD] Interpreting {func}(SetCond, ResetCond) -> set={set_var}, reset={reset_var}")

    set_tree = _build_fbd_tree(set_var, assigns).to_dict()
    reset_tree = _build_fbd_tree(reset_var, assigns).to_dict()

    return {
        "state": state_name,
        "assigned_from": d2_from,
        "rs_block": func,
        "set_var": set_var,
        "reset_var": reset_var,
        "set_tree": set_tree,
        "reset_tree": reset_tree,
    }


# -----------------------------
# 2) KG Traversal ohne SPARQL
# -----------------------------

def load_graph(ttl_path: str, trace: Optional[Tracer] = None) -> Graph:
    if trace:
        trace.log(f"[KG] Loading TTL: {ttl_path}")
    g = Graph()
    g.parse(ttl_path, format="turtle")
    if trace:
        trace.log(f"[KG] Loaded graph with {len(g)} triples")
    return g


def find_pou_by_name(graph: Graph, pou_name: str, trace: Optional[Tracer] = None) -> Optional[URIRef]:
    dp_hasPOUName = AG["dp_hasPOUName"]
    matches = list(graph.subjects(dp_hasPOUName, Literal(pou_name)))
    if trace:
        trace.log(f"[KG] find_pou_by_name('{pou_name}') -> {len(matches)} matches")
    return matches[0] if matches else None

from rdflib.namespace import RDF, XSD

def _literal_is_true(lit: Optional[Literal]) -> bool:
    if lit is None:
        return False
    # rdflib macht aus xsd:boolean oft Python bool
    try:
        if isinstance(lit.toPython(), bool):
            return bool(lit.toPython())
    except Exception:
        pass
    # fallback string
    s = str(lit).strip().lower()
    return s in ("true", "1", "yes")


def find_gemma_pous(graph: Graph, trace: Optional[Tracer] = None) -> List[Tuple[URIRef, str]]:
    """
    Entspricht SPARQL:
      ?pou a ag:class_CustomFBType ;
           ag:dp_isGEMMAStateMachine true .
      optional: ?pou ag:dp_hasPOUName ?name .
    Gibt Liste (pou_uri, pou_name) zurück, deterministisch sortiert nach Name.
    """
    dp_is_gemma = AG["dp_isGEMMAStateMachine"]
    dp_has_name = AG["dp_hasPOUName"]
    cls_custom = AG["class_CustomFBType"]

    hits: List[Tuple[URIRef, str]] = []

    for pou in graph.subjects(RDF.type, cls_custom):
        lit = next(graph.objects(pou, dp_is_gemma), None)
        if _literal_is_true(lit):
            name_lit = next(graph.objects(pou, dp_has_name), None)
            name = str(name_lit) if name_lit else str(pou)
            hits.append((pou, name))

    hits.sort(key=lambda x: x[1])  # deterministisch
    if trace:
        trace.log(f"[KG] GEMMA candidates (CustomFBType + isGEMMAStateMachine=true): {len(hits)}")
        for pou, name in hits[:10]:
            trace.log(f"[KG]   GEMMA POU: {name} -> {pou}")
    return hits

def get_pou_code(graph: Graph, pou_uri: URIRef, trace: Optional[Tracer] = None) -> Optional[str]:
    dp_hasPOUCode = AG["dp_hasPOUCode"]
    code_lit = next(graph.objects(pou_uri, dp_hasPOUCode), None)
    if trace:
        if code_lit:
            s = str(code_lit)
            trace.log(f"[KG] dp_hasPOUCode found for {pou_uri} (length={len(s)} chars)")
        else:
            trace.log(f"[KG] No dp_hasPOUCode for {pou_uri}")
    return str(code_lit) if code_lit else None


def get_port_by_name(graph: Graph, pou_uri: URIRef, port_name: str, trace: Optional[Tracer] = None) -> Optional[URIRef]:
    op_hasPort = AG["op_hasPort"]
    dp_hasPortName = AG["dp_hasPortName"]
    ports = list(graph.objects(pou_uri, op_hasPort))
    if trace:
        trace.log(f"[KG] POU {pou_uri} has {len(ports)} ports. Searching for port_name='{port_name}'")
    for port in ports:
        name = next(graph.objects(port, dp_hasPortName), None)
        if str(name) == port_name:
            if trace:
                trace.log(f"[KG] Found port '{port_name}' -> {port}")
            return port
    if trace:
        trace.log(f"[KG] Port '{port_name}' not found in POU {pou_uri}")
    return None


def resolve_input_to_upstream_output(
    graph: Graph,
    pou_uri: URIRef,
    input_port_name: str,
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    op_assignsToPort = AG["op_assignsToPort"]
    op_assignsFrom = AG["op_assignsFrom"]
    op_instantiatesPort = AG["op_instantiatesPort"]
    op_hasPort = AG["op_hasPort"]

    dp_hasPOUName = AG["dp_hasPOUName"]
    dp_hasPOUCode = AG["dp_hasPOUCode"]
    dp_hasPortName = AG["dp_hasPortName"]
    dp_hasPortDirection = AG["dp_hasPortDirection"]

    if trace:
        trace.log(f"[KG] Resolving input '{input_port_name}' in POU {pou_uri}")

    port = get_port_by_name(graph, pou_uri, input_port_name, trace=trace)
    if not port:
        return {"error": f"Port '{input_port_name}' nicht in POU {pou_uri} gefunden."}

    port_dir = next(graph.objects(port, dp_hasPortDirection), None)
    if trace:
        trace.log(f"[KG] Input port direction: {port_dir}")

    assigns = list(graph.subjects(op_assignsToPort, port))
    if trace:
        trace.log(f"[KG] op_assignsToPort count: {len(assigns)}")

    if not assigns:
        return {
            "pou_uri": str(pou_uri),
            "input_port": str(port),
            "input_port_dir": str(port_dir) if port_dir else None,
            "note": "Kein op_assignsToPort gefunden. Port könnte unverdrahtet oder aus Default kommen.",
        }

    assignment = assigns[0]
    if trace:
        trace.log(f"[KG] Using assignment: {assignment}")

    caller_port_instance = next(graph.objects(assignment, op_assignsFrom), None)
    if trace:
        trace.log(f"[KG] op_assignsFrom -> port_instance: {caller_port_instance}")
    if not caller_port_instance:
        return {"error": f"Assignment {assignment} hat kein op_assignsFrom."}

    caller_port = next(graph.objects(caller_port_instance, op_instantiatesPort), None)
    if trace:
        trace.log(f"[KG] port_instance op_instantiatesPort -> caller_port: {caller_port}")
    if not caller_port:
        return {"error": f"PortInstance {caller_port_instance} hat kein op_instantiatesPort."}

    caller_port_name = next(graph.objects(caller_port, dp_hasPortName), None)
    caller_port_dir = next(graph.objects(caller_port, dp_hasPortDirection), None)
    if trace:
        trace.log(f"[KG] caller_port_name={caller_port_name} caller_port_dir={caller_port_dir}")

    caller_pous = list(graph.subjects(op_hasPort, caller_port))
    if trace:
        trace.log(f"[KG] caller_pous owning caller_port: {len(caller_pous)}")
    if not caller_pous:
        return {"error": f"Kein CallerPOU gefunden, das caller_port {caller_port} besitzt."}

    caller_pou = caller_pous[0]
    caller_pou_name = next(graph.objects(caller_pou, dp_hasPOUName), None)
    caller_code = next(graph.objects(caller_pou, dp_hasPOUCode), None)
    if trace:
        trace.log(f"[KG] caller_pou={caller_pou} name={caller_pou_name}")
        trace.log(f"[KG] caller_code length={len(str(caller_code)) if caller_code else 0}")

    return {
        "pou_uri": str(pou_uri),
        "input_port_name": input_port_name,
        "input_port_dir": str(port_dir) if port_dir else None,
        "assignment": str(assignment),
        "caller_port_name": str(caller_port_name) if caller_port_name else None,
        "caller_port_dir": str(caller_port_dir) if caller_port_dir else None,
        "caller_pou_uri": str(caller_pou),
        "caller_pou_name": str(caller_pou_name) if caller_pou_name else None,
        "caller_code": str(caller_code) if caller_code else None,
    }

# -----------------------------
# 2b) GEMMA States + Auswahl-Logik für OR/AND per last_state
# -----------------------------

def gemma_state_list() -> List[str]:
    """
    Klassische GEMMA Zustände:
      A1..A7, F1..F6, D1..D3
    Quelle: diverse GEMMA-Unterlagen / Papers.
    """
    A = [f"A{i}" for i in range(1, 8)]
    F = [f"F{i}" for i in range(1, 7)]
    D = [f"D{i}" for i in range(1, 4)]
    return A + F + D


def _is_internal_v(var: str) -> bool:
    return bool(re.match(r"^V_\d+$", var))


def _collect_leaf_tokens(node: Dict[str, Any]) -> List[str]:
    """
    Sammelt alle leaf 'var' Tokens aus deinem to_dict()-Tree.
    """
    out: List[str] = []
    if "func" not in node:
        out.append(node["var"])
        return out
    for a in node.get("args", []) or []:
        out.extend(_collect_leaf_tokens(a))
    return out


def _collect_gemma_states_in_subtree(node: Dict[str, Any], gemma_states: set) -> set:
    leaves = _collect_leaf_tokens(node)
    return {x for x in leaves if x in gemma_states}


def _find_first_or_node(node: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """
    Findet den ersten OR_* Knoten in DFS-Reihenfolge.
    """
    func = node.get("func")
    if func and func.startswith("OR_"):
        return node
    for a in node.get("args", []) or []:
        found = _find_first_or_node(a)
        if found:
            return found
    return None


def _node_to_expr(node: Dict[str, Any]) -> str:
    """
    String-Rekonstruktion: AND_5(Auto_Stoerung, F1)
    """
    if "func" not in node:
        return node["var"]
    args = ", ".join(_node_to_expr(a) for a in (node.get("args") or []))
    return f"{node['func']}({args})"


def infer_suspected_input_port_from_last_state(
    d2_trace: Dict[str, Any],
    last_state: str,
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    """
    Erwartet d2_trace aus trace_d2_set_reset_from_fbd(...)
    Wählt deterministisch OR-Ast anhand last_state und gibt die weiter zu verfolgende Variable zurück.
    """
    gemma_states = set(gemma_state_list())

    if "error" in d2_trace:
        return {"error": d2_trace["error"]}

    set_tree = d2_trace.get("set_tree")
    if not set_tree:
        return {"error": "Kein set_tree im d2_trace vorhanden."}

    # 1) OR-Stelle finden
    or_node = _find_first_or_node(set_tree)
    if not or_node:
        return {
            "note": "Kein OR_* in set_tree gefunden. Keine automatische Branch-Auswahl implementiert.",
            "set_tree_expr": _node_to_expr(set_tree),
        }

    # 2) Alle OR-Äste ausgeben
    branches = or_node.get("args") or []
    branch_infos = []
    for idx, b in enumerate(branches, start=1):
        states_in_branch = _collect_gemma_states_in_subtree(b, gemma_states)
        branch_infos.append(
            {
                "idx": idx,
                "expr": _node_to_expr(b),
                "gemma_states_in_branch": sorted(states_in_branch),
                "contains_last_state": (last_state in states_in_branch),
            }
        )

    if trace:
        trace.log(f"[AUTO] Found OR node: {_node_to_expr(or_node)}")
        trace.log("[AUTO] OR branches:")
        for bi in branch_infos:
            trace.log(f"  - b{bi['idx']}: {bi['expr']}")
            trace.log(f"      states={bi['gemma_states_in_branch']} contains_last={bi['contains_last_state']}")

    # 3) deterministisch per last_state auswählen
    matching = [bi for bi in branch_infos if bi["contains_last_state"]]
    if len(matching) != 1:
        return {
            "or_branches": branch_infos,
            "error": f"Nicht deterministisch: {len(matching)} passende OR-Äste für last_state='{last_state}'.",
        }

    chosen = matching[0]
    chosen_node = branches[chosen["idx"] - 1]

    # 4) Kandidatenvariablen extrahieren:
    #    Leaf tokens im gewählten Ast minus GEMMA-States minus interne V_* (optional)
    leaves = _collect_leaf_tokens(chosen_node)
    candidates = []
    for tok in leaves:
        if tok in gemma_states:
            continue
        if _is_internal_v(tok):
            continue
        # kleine Filter: TRUE/FALSE o.ä. (falls auftauchen)
        if tok.upper() in ("TRUE", "FALSE"):
            continue
        candidates.append(tok)

    # uniq, deterministisch
    uniq: List[str] = []
    for c in candidates:
        if c not in uniq:
            uniq.append(c)

    if trace:
        trace.log(f"[AUTO] Chosen branch: {chosen['expr']}")
        trace.log(f"[AUTO] Next trace candidates (non-GEMMA leaves): {uniq}")

    return {
        "or_branches": branch_infos,
        "chosen_branch": chosen,
        "next_trace_candidates": uniq,
    }

# -----------------------------
# 3) ST Analyse: "var := TRUE;"
# -----------------------------

def extract_true_set_lines_st(
    st_code: str,
    var_name: str,
    with_line_numbers: bool = True,
    trace: Optional[Tracer] = None,
) -> List[Union[str, Tuple[int, str]]]:
    rx = re.compile(rf"^\s*{re.escape(var_name)}\s*:=\s*TRUE\s*;?\s*$")

    out: List[Union[str, Tuple[int, str]]] = []
    for i, line in enumerate(st_code.splitlines(), start=1):
        if rx.match(line):
            out.append((i, line) if with_line_numbers else line)

    if trace:
        trace.log(f"[ST] Searching TRUE assignments for '{var_name}' -> found {len(out)} lines")
        for item in out:
            if isinstance(item, tuple):
                trace.log(f"[ST] match line {item[0]}: {item[1].strip()}")
            else:
                trace.log(f"[ST] match: {item.strip()}")

    return out

# -----------------------------
# 3b) ST Analyse: IF-Bedingungen zu "var := TRUE;"
# -----------------------------

def _strip_st_comments(st: str) -> str:
    """
    Entfernt:
    - // line comments
    - (* block comments *)
    """
    st = re.sub(r"\(\*.*?\*\)", "", st, flags=re.S)
    st = re.sub(r"//.*?$", "", st, flags=re.M)
    return st


def extract_set_conditions_st(
    st_code: str,
    var_name: str,
    value_literal: str,  # "TRUE" oder "FALSE"
    trace: Optional[Tracer] = None,
    ) -> List[Dict[str, Any]]:
    """
    Findet Zeilen "var_name := <value_literal>;" und gibt die dazu aktiven IF-Bedingungen (Stack) zurück.
    """

    clean = _strip_st_comments(st_code)
    lines = clean.splitlines()

    rx_if    = re.compile(r"^\s*IF\s+(.*?)\s+THEN\s*$", flags=re.I)
    rx_elsif = re.compile(r"^\s*ELSIF\s+(.*?)\s+THEN\s*$", flags=re.I)
    rx_else  = re.compile(r"^\s*ELSE\s*$", flags=re.I)
    rx_end   = re.compile(r"^\s*END_IF\s*;?\s*$", flags=re.I)

    rx_assign = re.compile(
        rf"^\s*{re.escape(var_name)}\s*:=\s*{re.escape(value_literal)}\s*;\s*$",
        flags=re.I,
    )

    if_stack: List[Dict[str, Any]] = []
    results: List[Dict[str, Any]] = []

    def _stack_repr() -> List[str]:
        rep: List[str] = []
        for e in if_stack:
            if e["branch"] == "IF":
                rep.append(f"IF@{e['if_start_line']}: {e['cond']}")
            elif e["branch"] == "ELSIF":
                rep.append(f"ELSIF@{e['if_start_line']}: {e['cond']}")
            elif e["branch"] == "ELSE":
                rep.append(f"ELSE@{e['if_start_line']} (zu IF: {e['if_cond']})")
        return rep

    for i, raw in enumerate(lines, start=1):
        line = raw.strip()
        if not line:
            continue

        m_if = rx_if.match(line)
        if m_if:
            cond = m_if.group(1).strip()
            if_stack.append({"branch": "IF", "cond": cond, "if_start_line": i, "if_cond": cond})
            continue

        m_elsif = rx_elsif.match(line)
        if m_elsif:
            cond = m_elsif.group(1).strip()
            if not if_stack:
                if_stack.append({"branch": "ELSIF", "cond": cond, "if_start_line": i, "if_cond": cond})
            else:
                top = if_stack[-1]
                top["branch"] = "ELSIF"
                top["cond"] = cond
            continue

        if rx_else.match(line):
            if if_stack:
                top = if_stack[-1]
                top["branch"] = "ELSE"
                top["cond"] = ""
            continue

        if rx_end.match(line):
            if if_stack:
                if_stack.pop()
            continue

        if rx_assign.match(line):
            snapshot = _stack_repr()
            results.append(
                {
                    "line_no": i,
                    "assignment": raw.rstrip(),
                    "conditions": snapshot,
                    "conditions_conjunction": " AND ".join(snapshot) if snapshot else "(keine IF-Bedingung im Scope)",
                }
            )

    if trace:
        trace.log(f"[ST] IF-trace for '{var_name} := {value_literal}' -> {len(results)} hits")
        for r in results:
            trace.log(f"[ST] line {r['line_no']}: {r['assignment'].strip()}")
            trace.log(f"[ST]   conditions: {r['conditions_conjunction']}")

    return results

def analyze_var_assignments_st(
    st_code: str,
    var_name: str,
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    """
    Baut eine deterministische Liste aller Zuweisungen var := TRUE/FALSE mit IF-Pfaden.
    Zusätzlich: sortiert nach Zeilennummer und zeigt dir "last write wins"-Sicht.
    """

    true_hits = extract_set_conditions_st(st_code, var_name, "TRUE", trace=None)
    false_hits = extract_set_conditions_st(st_code, var_name, "FALSE", trace=None)

    # Merge + markieren
    merged: List[Dict[str, Any]] = []
    for h in true_hits:
        merged.append({**h, "value": "TRUE"})
    for h in false_hits:
        merged.append({**h, "value": "FALSE"})

    merged.sort(key=lambda x: x["line_no"])  # deterministisch nach Code-Reihenfolge

    # "Kann überschrieben werden?" rein syntaktisch: gibt es eine spätere Zuweisung?
    for idx, item in enumerate(merged):
        item["has_later_assignment"] = (idx < len(merged) - 1)
        item["later_assignments"] = [
            {"line_no": m["line_no"], "value": m["value"]}
            for m in merged[idx + 1 :]
        ]

    summary = {
        "var_name": var_name,
        "assignment_count": len(merged),
        "last_assignment_in_code": merged[-1] if merged else None,
    }

    if trace:
        trace.log(f"[ST] analyze_var_assignments_st('{var_name}') -> {len(merged)} assignments")
        for m in merged:
            trace.log(f"[ST]   line {m['line_no']}: {var_name} := {m['value']};")
            trace.log(f"[ST]     path: {m['conditions_conjunction']}")
        if merged:
            last = merged[-1]
            trace.log(f"[ST] last assignment in code: line {last['line_no']} -> {last['value']}")

    return {"summary": summary, "assignments": merged}


# -----------------------------
# 4) High-Level Helper mit Log Kette
# -----------------------------

def find_st_true_set_lines_for_d2_path(
    ttl_path: str,
    gemma_pou_name: str = "",     # kann leer bleiben -> auto
    state_name: str = "",
    suspected_input_port: str = "",
    last_gemma_state_before_failure: str = "",
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    if trace is None:
        trace = Tracer(enabled=True, print_live=True)

    trace.log("[RUN] Starting query chain")

    g = load_graph(ttl_path, trace=trace)

    # --- GEMMA POU dynamisch ermitteln ---
    if not gemma_pou_name:
        trace.log("[RUN] gemma_pou_name not provided -> auto-detect GEMMA POU via KG flags")
        gemma_candidates = find_gemma_pous(g, trace=trace)
        if not gemma_candidates:
            trace.log("[RUN] Abort: No GEMMA CustomFBType found")
            return {"error": "Kein GEMMA CustomFBType mit dp_isGEMMAStateMachine=true gefunden.", "trace_log": trace.lines}

        # deterministisch: nimm den ersten nach Name sortiert
        pou_uri, auto_name = gemma_candidates[0]
        gemma_pou_name = auto_name
        trace.log(f"[RUN] Selected GEMMA POU deterministically: {gemma_pou_name}")
    else:
        pou_uri = find_pou_by_name(g, gemma_pou_name, trace=trace)
        if not pou_uri:
            trace.log("[RUN] Abort: GEMMA POU not found by name")
            return {"error": f"Konnte GEMMA POU '{gemma_pou_name}' nicht im KG finden.", "trace_log": trace.lines}

    fbd_code = get_pou_code(g, pou_uri, trace=trace)
    if not fbd_code:
        trace.log("[RUN] Abort: No dp_hasPOUCode for GEMMA POU")
        return {"error": f"Konnte dp_hasPOUCode für '{gemma_pou_name}' nicht lesen.", "trace_log": trace.lines}

    trace.log(f"[RUN] Tracing state '{state_name}' in FBD Python export")
    d2_trace = trace_d2_set_reset_from_fbd(fbd_code, state_name=state_name, trace=trace)

    if not last_gemma_state_before_failure:
        return {
            "d2_trace": d2_trace,
            "error": "last_gemma_state_before_failure ist leer. Bitte übergeben (z.B. 'F1').",
            "trace_log": trace.lines,
        }

    auto = infer_suspected_input_port_from_last_state(
        d2_trace,
        last_state=last_gemma_state_before_failure,
        trace=trace,
    )

    if "error" in auto:
        return {"d2_trace": d2_trace, "auto_port": auto, "error": auto["error"], "trace_log": trace.lines}

    cands = auto.get("next_trace_candidates", [])
    if not cands:
        return {"d2_trace": d2_trace, "auto_port": auto, "error": "Keine Kandidatenvariable gefunden.", "trace_log": trace.lines}

    suspected_input_port = cands[0]  # deterministisch: erster Kandidat
    trace.log(f"[RUN] Auto-detected suspected_input_port = {suspected_input_port}")

    trace.log(f"[RUN] Resolving input '{suspected_input_port}' upstream in KG")
    origin = resolve_input_to_upstream_output(g, pou_uri, suspected_input_port, trace=trace)

    if "caller_code" not in origin or not origin.get("caller_code"):
        trace.log("[RUN] Abort: No caller_code found for upstream resolution")
        return {"d2_trace": d2_trace, "origin": origin, "error": "Kein caller_code gefunden.", "trace_log": trace.lines}

    caller_port_name = origin.get("caller_port_name")
    if not caller_port_name:
        trace.log("[RUN] Abort: caller_port_name missing from origin")
        return {
            "d2_trace": d2_trace,
            "origin": origin,
            "error": "caller_port_name fehlt. Kann keine ST-Assignments suchen.",
            "trace_log": trace.lines,
        }
    
    st_code = origin["caller_code"]

    trace.log(f"[RUN] Extracting TRUE set lines in caller ST for variable '{caller_port_name}'")
    true_lines = extract_true_set_lines_st(st_code, caller_port_name, with_line_numbers=True, trace=trace)

    trace.log(f"[RUN] Extracting IF-conditions leading to '{caller_port_name} := TRUE;'")
    true_conditions = extract_set_conditions_st(
        st_code,
        var_name=caller_port_name,
        value_literal="TRUE",
        trace=trace,
    )

    trace.log(f"[RUN] Extracting IF-conditions leading to '{caller_port_name} := FALSE;'")
    false_conditions = extract_set_conditions_st(
        st_code,
        var_name=caller_port_name,
        value_literal="FALSE",
        trace=trace,
    )
    trace.log(f"[RUN] Building merged assignment order for '{caller_port_name}' (TRUE/FALSE + last-write view)")
    assignment_analysis = analyze_var_assignments_st(
        st_code,
        var_name=caller_port_name,
        trace=trace,
    )

    trace.log("[RUN] Done")

    return {
        "gemma_pou_name": gemma_pou_name,
        "d2_trace": d2_trace,
        "origin": origin,
        "true_set_lines": true_lines,
        "true_set_conditions": true_conditions,
        "trace_log": trace.lines,
        "last_gemma_state_before_failure": last_gemma_state_before_failure,
        "auto_port": auto,
        "suspected_input_port": suspected_input_port,
        "false_set_conditions": false_conditions,
        "assignment_analysis": assignment_analysis,
    }


# -----------------------------
# Ausführung
# -----------------------------

tr = Tracer(enabled=True, print_live=True)

result = find_st_true_set_lines_for_d2_path(
    ttl_path=r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl",
    gemma_pou_name="",   # auto
    state_name="D2",
    suspected_input_port="",  # wird ja auto-detected
    last_gemma_state_before_failure="F1",  # <-- HIER
    trace=tr,
)

print("\n=== TRUE Set Lines ===")
for item in result.get("true_set_lines", []):
    print(item)

print("\n=== TRUE Set Conditions (IF paths) ===")
for r in result.get("true_set_conditions", []):
    print(f"Line {r['line_no']}: {r['assignment'].strip()}")
    print(f"  Conditions: {r['conditions_conjunction']}")

print("\n=== FALSE Set Conditions (IF paths) ===")
for r in result.get("false_set_conditions", []):
    print(f"Line {r['line_no']}: {r['assignment'].strip()}")
    print(f"  Conditions: {r['conditions_conjunction']}")

print("\n=== Assignment order (merged TRUE/FALSE) ===")
analysis = result.get("assignment_analysis", {})
for a in analysis.get("assignments", []):
    print(f"Line {a['line_no']}: {result['origin']['caller_port_name']} := {a['value']};")
    print(f"  Path: {a['conditions_conjunction']}")
    if a["has_later_assignment"]:
        print(f"  Can be overwritten later by: {a['later_assignments']}")


[RUN] Starting query chain
[KG] Loading TTL: D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl
[KG] Loaded graph with 2530 triples
[RUN] gemma_pou_name not provided -> auto-detect GEMMA POU via KG flags
[KG] GEMMA candidates (CustomFBType + isGEMMAStateMachine=true): 1
[KG]   GEMMA POU: FB_Betriebsarten -> http://www.semanticweb.org/AgentProgramParams/FBType_FB_Betriebsarten
[RUN] Selected GEMMA POU deterministically: FB_Betriebsarten
[KG] dp_hasPOUCode found for http://www.semanticweb.org/AgentProgramParams/FBType_FB_Betriebsarten (length=6448 chars)
[RUN] Tracing state 'D2' in FBD Python export
[FBD] Parsed call assignments: 35 entries (V_* = FUNC(args...))
[FBD] Found final state assignment: D2 = V_40000000017
[FBD] V_40000000017 is computed by RS_4 with args ['V_40000000010', 'V_40000000016']
[FBD] Interpreting RS_4(SetCond, ResetCond) -> set=V_40000000010, reset=V_40000000016
[AUTO] Found OR node: OR_5(AND_5(Auto_Stoerung, F1), AND_6(GVL.DiagnoseRequested, D1, GVL.NotStopp)

In [2]:
from __future__ import annotations

import re
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, Set

from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS

AG = Namespace("http://www.semanticweb.org/AgentProgramParams/")

# ---------
# Config
# ---------
TTL_PATH = r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl"  # <--- dein Pfad
FBTYPE_URI = URIRef("http://www.semanticweb.org/AgentProgramParams/FBType_FB_Automatikbetrieb_F1")
START_EXPR = "pPer.Q"  # <--- darauf fokussieren wir uns

# ---------
# Load KG + ST code
# ---------
g = Graph()
g.parse(TTL_PATH, format="turtle")

st_code_lit = next(g.objects(FBTYPE_URI, AG["dp_hasPOUCode"]), None)
if not st_code_lit:
    raise RuntimeError(f"Kein dp_hasPOUCode für {FBTYPE_URI} gefunden.")
ST_CODE = str(st_code_lit)


# -----------------------------
# Helpers: KG
# -----------------------------
def _first(obj_iter):
    return next(iter(obj_iter), None)

def find_internal_variable_uri(fbtype_uri: URIRef, var_name: str) -> Optional[URIRef]:
    """Findet Var_* im FBType über op_hasInternalVariable/op_usesVariable + dp_hasVariableName=var_name."""
    for pred in (AG["op_hasInternalVariable"], AG["op_usesVariable"]):
        for v in g.objects(fbtype_uri, pred):
            name = _first(g.objects(v, AG["dp_hasVariableName"]))
            if name and str(name) == var_name:
                return v
    return None

def describe_internal_variable(fbtype_uri: URIRef, var_name: str) -> Dict[str, Any]:
    """
    pPer -> Var_* -> dp_hasVariableType (z.B. TP)
         -> op_representsFBInstance -> FBInst_*
         -> op_isInstanceOfFBType -> StandardFBType_TP
         -> rdfs:comment -> Beschreibung
    """
    var_uri = find_internal_variable_uri(fbtype_uri, var_name)
    if not var_uri:
        return {"error": f"Interne Variable '{var_name}' nicht gefunden (kein Var_* via op_hasInternalVariable/op_usesVariable)."}

    var_type = _first(g.objects(var_uri, AG["dp_hasVariableType"]))
    fbinst = _first(g.objects(var_uri, AG["op_representsFBInstance"]))
    fbinst_type = _first(g.objects(fbinst, AG["op_isInstanceOfFBType"])) if fbinst else None
    descr = _first(g.objects(fbinst_type, RDFS.comment)) if fbinst_type else None

    return {
        "var_name": var_name,
        "var_uri": str(var_uri),
        "var_type": str(var_type) if var_type else None,
        "fbinst_uri": str(fbinst) if fbinst else None,
        "fbinst_type_uri": str(fbinst_type) if fbinst_type else None,
        "fbinst_type_comment": str(descr) if descr else None,
    }

def find_port_uri(fbtype_uri: URIRef, port_name: str) -> Optional[URIRef]:
    for p in g.objects(fbtype_uri, AG["op_hasPort"]):
        n = _first(g.objects(p, AG["dp_hasPortName"]))
        if n and str(n) == port_name:
            return p
    return None

def port_default_and_wiring(fbtype_uri: URIRef, port_name: str) -> Dict[str, Any]:
    port_uri = find_port_uri(fbtype_uri, port_name)
    if not port_uri:
        return {"note": f"'{port_name}' ist kein Port von {fbtype_uri}."}

    default_val = _first(g.objects(port_uri, AG["dp_hasDefaultPortValue"]))
    assigns = list(g.subjects(AG["op_assignsToPort"], port_uri))

    return {
        "port_name": port_name,
        "port_uri": str(port_uri),
        "default_value": str(default_val) if default_val else None,
        "is_wired": bool(assigns),
        "assignments_count": len(assigns),
    }


# -----------------------------
# Helpers: ST parse (FB Calls)
# -----------------------------
def _split_top_level_commas(s: str) -> List[str]:
    parts = []
    buf = []
    depth = 0
    in_str = False

    i = 0
    while i < len(s):
        ch = s[i]
        if ch == "'" and (i == 0 or s[i-1] != "\\"):
            in_str = not in_str
            buf.append(ch)
        elif not in_str:
            if ch == "(":
                depth += 1
                buf.append(ch)
            elif ch == ")":
                depth = max(0, depth - 1)
                buf.append(ch)
            elif ch == "," and depth == 0:
                parts.append("".join(buf).strip())
                buf = []
            else:
                buf.append(ch)
        else:
            buf.append(ch)
        i += 1

    tail = "".join(buf).strip()
    if tail:
        parts.append(tail)
    return parts

def find_fb_call_lines(st_code: str, inst_name: str) -> List[Tuple[int, str]]:
    rx = re.compile(rf"^\s*{re.escape(inst_name)}\s*\((.*?)\)\s*;\s*$")
    hits = []
    for i, line in enumerate(st_code.splitlines(), start=1):
        m = rx.match(line)
        if m:
            hits.append((i, line.rstrip()))
    return hits

def parse_fb_call_args(call_line: str, inst_name: str) -> Dict[str, str]:
    # call_line z.B. "pPer(IN := rPer.Q, PT := FaultPulse);"
    m = re.match(rf"^\s*{re.escape(inst_name)}\s*\((.*?)\)\s*;\s*$", call_line)
    if not m:
        return {}
    argstr = m.group(1).strip()
    if not argstr:
        return {}

    args = _split_top_level_commas(argstr)
    mapping: Dict[str, str] = {}
    for a in args:
        # unterstützt ":=" und "=>"
        m_in = re.match(r"^\s*([A-Za-z_]\w*)\s*:=\s*(.+?)\s*$", a)
        m_out = re.match(r"^\s*([A-Za-z_]\w*)\s*=>\s*(.+?)\s*$", a)
        if m_in:
            mapping[m_in.group(1)] = m_in.group(2).strip()
        elif m_out:
            mapping[m_out.group(1)] = m_out.group(2).strip()
    return mapping

def extract_dotted_tokens(expr: str) -> List[str]:
    # findet "rPer.Q", "tPer.Q" etc.
    return re.findall(r"\b[A-Za-z_]\w*\.[A-Za-z_]\w*\b", expr)


# -----------------------------
# Trace Engine: expr wie "pPer.Q"
# -----------------------------
@dataclass
class TraceLine:
    indent: int
    text: str

def trace_expr(expr: str, fbtype_uri: URIRef, st_code: str, *, max_depth: int = 10) -> List[TraceLine]:
    lines: List[TraceLine] = []
    visited: Set[str] = set()

    def rec(e: str, depth: int):
        key = e.strip()
        if key in visited:
            lines.append(TraceLine(depth, f"(loop detected) {key}"))
            return
        visited.add(key)

        if depth > max_depth:
            lines.append(TraceLine(depth, f"(max depth) {key}"))
            return

        # dotted?
        if "." in key:
            inst, port = key.split(".", 1)
            desc = describe_internal_variable(fbtype_uri, inst)
            if "error" in desc:
                lines.append(TraceLine(depth, f"{key}: {desc['error']}"))
                return

            lines.append(TraceLine(depth, f"{key} ist Port '{port}' der FB-Instanz '{inst}' (Typ={desc['var_type']})"))
            if desc.get("fbinst_type_comment"):
                lines.append(TraceLine(depth + 1, f"FBType-Beschreibung: {desc['fbinst_type_comment']}"))

            call_hits = find_fb_call_lines(st_code, inst)
            if not call_hits:
                lines.append(TraceLine(depth + 1, f"Keine Call-Line '{inst}(...)' im ST-Code gefunden."))
                return

            # deterministisch: nimm erste Call-Line
            call_line_no, call_line = call_hits[0]
            lines.append(TraceLine(depth + 1, f"Call-Line @ {call_line_no}: {call_line.strip()}"))
            args = parse_fb_call_args(call_line, inst)

            # Für Output Q können wir nicht "ausrechnen", aber wir können die Eingänge zeigen
            if args:
                for k_arg, v_expr in args.items():
                    lines.append(TraceLine(depth + 2, f"Param {k_arg} = {v_expr}"))
                    for dt in extract_dotted_tokens(v_expr):
                        rec(dt, depth + 3)

                    # plain identifier: prüfen ob es Input-Port des aktuellen FB ist, dann Default/Wiring ausgeben
                    plain = v_expr.strip()
                    if re.fullmatch(r"[A-Za-z_]\w*", plain):
                        port_info = port_default_and_wiring(fbtype_uri, plain)
                        if "port_uri" in port_info:
                            dv = port_info.get("default_value")
                            wired = port_info.get("is_wired")
                            lines.append(TraceLine(depth + 3, f"'{plain}' ist Input-Port von FB_Automatikbetrieb_F1"))
                            lines.append(TraceLine(depth + 4, f"default_value = {dv}"))
                            lines.append(TraceLine(depth + 4, f"is_wired = {wired} (assignments={port_info.get('assignments_count')})"))
                            if not wired:
                                lines.append(TraceLine(depth + 4, "=> nicht verdrahtet, daher wird DefaultValue verwendet (Trace endet hier)."))
            return

        # plain identifier: Port Default/Wiring check
        if re.fullmatch(r"[A-Za-z_]\w*", key):
            port_info = port_default_and_wiring(fbtype_uri, key)
            if "port_uri" in port_info:
                dv = port_info.get("default_value")
                wired = port_info.get("is_wired")
                lines.append(TraceLine(depth, f"'{key}' ist Input-Port von FB_Automatikbetrieb_F1"))
                lines.append(TraceLine(depth + 1, f"default_value = {dv}"))
                lines.append(TraceLine(depth + 1, f"is_wired = {wired} (assignments={port_info.get('assignments_count')})"))
                if not wired:
                    lines.append(TraceLine(depth + 1, "=> nicht verdrahtet, daher wird DefaultValue verwendet (Trace endet hier)."))
            else:
                lines.append(TraceLine(depth, f"'{key}' ist kein (erkennbare) Input-Port hier."))
            return

        lines.append(TraceLine(depth, f"Unbekanntes Token: {key}"))

    rec(expr, 0)
    return lines


# -----------------------------
# Run trace for pPer.Q
# -----------------------------
trace_lines = trace_expr(START_EXPR, FBTYPE_URI, ST_CODE, max_depth=10)

print("=== TRACE START ===")
for tl in trace_lines:
    print("  " * tl.indent + tl.text)
print("=== TRACE END ===")


=== TRACE START ===
pPer.Q ist Port 'Q' der FB-Instanz 'pPer' (Typ=TP)
  FBType-Beschreibung: TP ist ein Impulsbaustein. Eine steigende Flanke an IN startet einen Impuls. Q wird für genau die Dauer PT TRUE, unabhängig davon wie lange IN TRUE bleibt. ET zeigt die verstrichene Impulszeit.
  Call-Line @ 74: pPer(IN := rPer.Q, PT := FaultPulse);
    Param IN = rPer.Q
      rPer.Q ist Port 'Q' der FB-Instanz 'rPer' (Typ=R_TRIG)
        FBType-Beschreibung: R_TRIG erkennt eine steigende Flanke an CLK. Q wird genau für einen Programmscan TRUE, wenn CLK von FALSE auf TRUE wechselt. Damit lassen sich aus einem Signal einzelne Ereignisimpulse ableiten.
        Call-Line @ 73: rPer(CLK := tPer.Q);
          Param CLK = tPer.Q
            tPer.Q ist Port 'Q' der FB-Instanz 'tPer' (Typ=TON)
              FBType-Beschreibung: TON ist eine Einschaltverzögerung. Wenn IN TRUE wird, startet die Zeitmessung. ET zählt die verstrichene Zeit hoch. Sobald ET die Vorgabezeit PT erreicht und IN weiterhin TRUE 

In [ ]:
from __future__ import annotations

import re
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, Union, Set, Callable

from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, XSD

AG = Namespace("http://www.semanticweb.org/AgentProgramParams/")


# -----------------------------
# Tracer
# -----------------------------
@dataclass
class Tracer:
    enabled: bool = True
    print_live: bool = True
    lines: List[str] = None

    def __post_init__(self):
        if self.lines is None:
            self.lines = []

    def log(self, msg: str) -> None:
        if not self.enabled:
            return
        self.lines.append(msg)
        if self.print_live:
            print(msg)


# ==========================================
# TEIL 1: DEIN ORIGINAL-CODE (FBD & ST Analyse)
# ==========================================

@dataclass
class FbdNode:
    var: str
    func: Optional[str] = None
    args: Optional[List["FbdNode"]] = None

    def to_dict(self) -> Dict[str, Any]:
        if not self.func:
            return {"var": self.var}
        return {"var": self.var, "func": self.func, "args": [a.to_dict() for a in (self.args or [])]}

def _parse_fbd_call_assignments(fbd_py_code: str, trace: Optional[Tracer] = None) -> Dict[str, Tuple[str, List[str]]]:
    pattern = r"^\s*(V_\d+)\s*=\s*([A-Za-z_][A-Za-z0-9_]*)\(([^)]*)\)\s*$"
    m: Dict[str, Tuple[str, List[str]]] = {}
    for mm in re.finditer(pattern, fbd_py_code, flags=re.M):
        var, func, argstr = mm.group(1), mm.group(2), mm.group(3)
        args = [a.strip() for a in argstr.split(",") if a.strip()]
        m[var] = (func, args)
    return m

def _find_final_state_assignment_var(fbd_py_code: str, state_name: str = "D2", trace: Optional[Tracer] = None) -> Optional[str]:
    m = re.search(rf"^\s*{re.escape(state_name)}\s*=\s*(V_\d+)\s*$", fbd_py_code, flags=re.M)
    return m.group(1) if m else None

def _build_fbd_tree(var: str, assigns: Dict[str, Tuple[str, List[str]]], depth: int = 0, max_depth: int = 25) -> FbdNode:
    if depth >= max_depth: return FbdNode(var=var, func="MAX_DEPTH", args=[])
    if var not in assigns: return FbdNode(var=var)
    func, args = assigns[var]
    return FbdNode(var=var, func=func, args=[_build_fbd_tree(a, assigns, depth + 1, max_depth) for a in args])

def trace_d2_set_reset_from_fbd(fbd_py_code: str, state_name: str = "D2", trace: Optional[Tracer] = None) -> Dict[str, Any]:
    assigns = _parse_fbd_call_assignments(fbd_py_code, trace=trace)
    d2_from = _find_final_state_assignment_var(fbd_py_code, state_name=state_name, trace=trace)
    if not d2_from: return {"error": f"Konnte '{state_name} = V_...' im Code nicht finden."}
    if d2_from not in assigns: return {"error": f"'{state_name}' kommt von '{d2_from}', aber '{d2_from}' hat keine Call-Zuweisung im Code."}

    func, args = assigns[d2_from]
    if not (func.startswith("RS_") and len(args) >= 2):
        full_tree = _build_fbd_tree(d2_from, assigns).to_dict()
        return {"state": state_name, "assigned_from": d2_from, "tree": full_tree}

    set_var, reset_var = args[0], args[1]
    return {
        "state": state_name, "assigned_from": d2_from, "rs_block": func,
        "set_var": set_var, "reset_var": reset_var,
        "set_tree": _build_fbd_tree(set_var, assigns).to_dict(),
        "reset_tree": _build_fbd_tree(reset_var, assigns).to_dict(),
    }

def load_graph(ttl_path: str, trace: Optional[Tracer] = None) -> Graph:
    g = Graph()
    g.parse(ttl_path, format="turtle")
    return g

def find_pou_by_name(graph: Graph, pou_name: str, trace: Optional[Tracer] = None) -> Optional[URIRef]:
    matches = list(graph.subjects(AG["dp_hasPOUName"], Literal(pou_name)))
    return matches[0] if matches else None

def _literal_is_true(lit: Optional[Literal]) -> bool:
    if lit is None: return False
    try:
        if isinstance(lit.toPython(), bool): return bool(lit.toPython())
    except Exception: pass
    return str(lit).strip().lower() in ("true", "1", "yes")

def find_gemma_pous(graph: Graph, trace: Optional[Tracer] = None) -> List[Tuple[URIRef, str]]:
    """
    Entspricht SPARQL:
      ?pou a ag:class_CustomFBType ;
           ag:dp_isGEMMAStateMachine true .
      optional: ?pou ag:dp_hasPOUName ?name .
    Gibt Liste (pou_uri, pou_name) zurück, deterministisch sortiert nach Name.
    """
    dp_is_gemma = AG["dp_isGEMMAStateMachine"]
    dp_has_name = AG["dp_hasPOUName"]
    cls_custom = AG["class_CustomFBType"]

    hits: List[Tuple[URIRef, str]] = []
    for pou in graph.subjects(RDF.type, cls_custom):
        lit = next(graph.objects(pou, dp_is_gemma), None)
        if _literal_is_true(lit):
            name_lit = next(graph.objects(pou, dp_has_name), None)
            name = str(name_lit) if name_lit else str(pou)
            hits.append((pou, name))

    hits.sort(key=lambda x: x[1])  # deterministisch
    if trace:
        trace.log(f"[KG] GEMMA candidates (CustomFBType + isGEMMAStateMachine=true): {len(hits)}")
        for pou, name in hits[:10]:
            trace.log(f"[KG]   GEMMA POU: {name} -> {pou}")
    return hits

def get_pou_code(graph: Graph, pou_uri: URIRef, trace: Optional[Tracer] = None) -> Optional[str]:
    dp_hasPOUCode = AG["dp_hasPOUCode"]
    code_lit = next(graph.objects(pou_uri, dp_hasPOUCode), None)
    if trace:
        if code_lit:
            s = str(code_lit)
            trace.log(f"[KG] dp_hasPOUCode found for {pou_uri} (length={len(s)} chars)")
        else:
            trace.log(f"[KG] No dp_hasPOUCode for {pou_uri}")
    return str(code_lit) if code_lit else None

def get_port_by_name(graph: Graph, pou_uri: URIRef, port_name: str, trace: Optional[Tracer] = None) -> Optional[URIRef]:
    op_hasPort = AG["op_hasPort"]
    dp_hasPortName = AG["dp_hasPortName"]
    ports = list(graph.objects(pou_uri, op_hasPort))
    if trace:
        trace.log(f"[KG] POU {pou_uri} has {len(ports)} ports. Searching for port_name='{port_name}'")
    for port in ports:
        name = next(graph.objects(port, dp_hasPortName), None)
        if str(name) == port_name:
            if trace:
                trace.log(f"[KG] Found port '{port_name}' -> {port}")
            return port
    if trace:
        trace.log(f"[KG] Port '{port_name}' not found in POU {pou_uri}")
    return None

def resolve_input_to_upstream_output(graph: Graph, pou_uri: URIRef, input_port_name: str, trace: Optional[Tracer] = None) -> Dict[str, Any]:
    op_assignsToPort = AG["op_assignsToPort"]
    op_assignsFrom = AG["op_assignsFrom"]
    op_instantiatesPort = AG["op_instantiatesPort"]
    op_hasPort = AG["op_hasPort"]

    dp_hasPOUName = AG["dp_hasPOUName"]
    dp_hasPOUCode = AG["dp_hasPOUCode"]
    dp_hasPortName = AG["dp_hasPortName"]
    dp_hasPortDirection = AG["dp_hasPortDirection"]

    if trace:
        trace.log(f"[KG] Resolving input '{input_port_name}' in POU {pou_uri}")

    port = get_port_by_name(graph, pou_uri, input_port_name, trace=trace)
    if not port:
        return {"error": f"Port '{input_port_name}' nicht in POU {pou_uri} gefunden."}

    port_dir = next(graph.objects(port, dp_hasPortDirection), None)
    if trace:
        trace.log(f"[KG] Input port direction: {port_dir}")

    assigns = list(graph.subjects(op_assignsToPort, port))
    if trace:
        trace.log(f"[KG] op_assignsToPort count: {len(assigns)}")

    if not assigns:
        return {
            "pou_uri": str(pou_uri),
            "input_port": str(port),
            "input_port_dir": str(port_dir) if port_dir else None,
            "note": "Kein op_assignsToPort gefunden. Port könnte unverdrahtet oder aus Default kommen.",
        }

    assignment = assigns[0]
    if trace:
        trace.log(f"[KG] Using assignment: {assignment}")

    caller_port_instance = next(graph.objects(assignment, op_assignsFrom), None)
    if trace:
        trace.log(f"[KG] op_assignsFrom -> port_instance: {caller_port_instance}")
    if not caller_port_instance:
        return {"error": f"Assignment {assignment} hat kein op_assignsFrom."}

    caller_port = next(graph.objects(caller_port_instance, op_instantiatesPort), None)
    if trace:
        trace.log(f"[KG] port_instance op_instantiatesPort -> caller_port: {caller_port}")
    if not caller_port:
        return {"error": f"PortInstance {caller_port_instance} hat kein op_instantiatesPort."}

    caller_port_name = next(graph.objects(caller_port, dp_hasPortName), None)
    caller_port_dir = next(graph.objects(caller_port, dp_hasPortDirection), None)
    if trace:
        trace.log(f"[KG] caller_port_name={caller_port_name} caller_port_dir={caller_port_dir}")

    caller_pous = list(graph.subjects(op_hasPort, caller_port))
    if trace:
        trace.log(f"[KG] caller_pous owning caller_port: {len(caller_pous)}")
    if not caller_pous:
        return {"error": f"Kein CallerPOU gefunden, das caller_port {caller_port} besitzt."}

    caller_pou = caller_pous[0]
    caller_pou_name = next(graph.objects(caller_pou, dp_hasPOUName), None)
    caller_code = next(graph.objects(caller_pou, dp_hasPOUCode), None)
    if trace:
        trace.log(f"[KG] caller_pou={caller_pou} name={caller_pou_name}")
        trace.log(f"[KG] caller_code length={len(str(caller_code)) if caller_code else 0}")

    return {
        "pou_uri": str(pou_uri),
        "input_port_name": input_port_name,
        "input_port_dir": str(port_dir) if port_dir else None,
        "assignment": str(assignment),
        "caller_port_name": str(caller_port_name) if caller_port_name else None,
        "caller_port_dir": str(caller_port_dir) if caller_port_dir else None,
        "caller_pou_uri": str(caller_pou),
        "caller_pou_name": str(caller_pou_name) if caller_pou_name else None,
        "caller_code": str(caller_code) if caller_code else None,
    }

def gemma_state_list() -> List[str]:
    return [f"A{i}" for i in range(1, 8)] + [f"F{i}" for i in range(1, 7)] + [f"D{i}" for i in range(1, 4)]

def _is_internal_v(var: str) -> bool: return bool(re.match(r"^V_\d+$", var))

def _collect_leaf_tokens(node: Dict[str, Any]) -> List[str]:
    if "func" not in node: return [node["var"]]
    out = []
    for a in node.get("args", []) or []: out.extend(_collect_leaf_tokens(a))
    return out

def _collect_gemma_states_in_subtree(node: Dict[str, Any], gemma_states: set) -> set:
    return {x for x in _collect_leaf_tokens(node) if x in gemma_states}

def _find_first_or_node(node: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    if node.get("func") and node["func"].startswith("OR_"): return node
    for a in node.get("args", []) or []:
        if found := _find_first_or_node(a): return found
    return None

def _node_to_expr(node: Dict[str, Any]) -> str:
    if "func" not in node: return node["var"]
    return f"{node['func']}({', '.join(_node_to_expr(a) for a in (node.get('args') or []))})"

def infer_suspected_input_port_from_last_state(
    d2_trace: Dict[str, Any],
    last_state: str,
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    """
    Erwartet d2_trace aus trace_d2_set_reset_from_fbd(...)
    W?hlt deterministisch OR-Ast anhand last_state und gibt die weiter zu verfolgende Variable zur?ck.
    """
    gemma_states = set(gemma_state_list())

    if "error" in d2_trace:
        return {"error": d2_trace["error"]}

    set_tree = d2_trace.get("set_tree")
    if not set_tree:
        return {"error": "Kein set_tree im d2_trace vorhanden."}

    # 1) OR-Stelle finden
    or_node = _find_first_or_node(set_tree)
    if not or_node:
        return {
            "note": "Kein OR_* in set_tree gefunden. Keine automatische Branch-Auswahl implementiert.",
            "set_tree_expr": _node_to_expr(set_tree),
        }

    # 2) Alle OR-?ste ausgeben
    branches = or_node.get("args") or []
    branch_infos = []
    for idx, b in enumerate(branches, start=1):
        states_in_branch = _collect_gemma_states_in_subtree(b, gemma_states)
        branch_infos.append(
            {
                "idx": idx,
                "expr": _node_to_expr(b),
                "gemma_states_in_branch": sorted(states_in_branch),
                "contains_last_state": (last_state in states_in_branch),
            }
        )

    if trace:
        trace.log(f"[AUTO] Found OR node: {_node_to_expr(or_node)}")
        trace.log("[AUTO] OR branches:")
        for bi in branch_infos:
            trace.log(f"  - b{bi['idx']}: {bi['expr']}")
            trace.log(f"      states={bi['gemma_states_in_branch']} contains_last={bi['contains_last_state']}")

    # 3) deterministisch per last_state ausw?hlen
    matching = [bi for bi in branch_infos if bi["contains_last_state"]]
    if len(matching) != 1:
        return {
            "or_branches": branch_infos,
            "error": f"Nicht deterministisch: {len(matching)} passende OR-?ste f?r last_state='{last_state}'.",
        }

    chosen = matching[0]
    chosen_node = branches[chosen["idx"] - 1]

    # 4) Kandidatenvariablen extrahieren
    leaves = _collect_leaf_tokens(chosen_node)
    candidates = []
    for tok in leaves:
        if tok in gemma_states:
            continue
        if _is_internal_v(tok):
            continue
        if tok.upper() in ("TRUE", "FALSE"):
            continue
        candidates.append(tok)

    # uniq, deterministisch
    uniq: List[str] = []
    for c in candidates:
        if c not in uniq:
            uniq.append(c)

    if trace:
        trace.log(f"[AUTO] Chosen branch: {chosen['expr']}")
        trace.log(f"[AUTO] Next trace candidates (non-GEMMA leaves): {uniq}")

    return {
        "or_branches": branch_infos,
        "chosen_branch": chosen,
        "next_trace_candidates": uniq,
    }


def extract_true_set_lines_st(
    st_code: str,
    var_name: str,
    with_line_numbers: bool = True,
    trace: Optional[Tracer] = None,
) -> List[Union[str, Tuple[int, str]]]:
    rx = re.compile(rf"^\s*{re.escape(var_name)}\s*:=\s*TRUE\s*;?\s*$")

    out: List[Union[str, Tuple[int, str]]] = []
    for i, line in enumerate(st_code.splitlines(), start=1):
        if rx.match(line):
            out.append((i, line) if with_line_numbers else line)

    if trace:
        trace.log(f"[ST] Searching TRUE assignments for '{var_name}' -> found {len(out)} lines")
        for item in out:
            if isinstance(item, tuple):
                trace.log(f"[ST] match line {item[0]}: {item[1].strip()}")
            else:
                trace.log(f"[ST] match: {item.strip()}")

    return out


def _strip_st_comments(st: str) -> str:
    st = re.sub(r"\(\*.*?\*\)", "", st, flags=re.S)
    return re.sub(r"//.*?$", "", st, flags=re.M)

def extract_set_conditions_st(
    st_code: str,
    var_name: str,
    value_literal: str,  # "TRUE" oder "FALSE"
    trace: Optional[Tracer] = None,
) -> List[Dict[str, Any]]:
    """
    Findet Zeilen "var_name := <value_literal>;" und gibt die dazu aktiven IF-Bedingungen (Stack) zur?ck.
    """

    clean = _strip_st_comments(st_code)
    lines = clean.splitlines()

    rx_if    = re.compile(r"^\s*IF\s+(.*?)\s+THEN\s*$", flags=re.I)
    rx_elsif = re.compile(r"^\s*ELSIF\s+(.*?)\s+THEN\s*$", flags=re.I)
    rx_else  = re.compile(r"^\s*ELSE\s*$", flags=re.I)
    rx_end   = re.compile(r"^\s*END_IF\s*;?\s*$", flags=re.I)

    rx_assign = re.compile(
        rf"^\s*{re.escape(var_name)}\s*:=\s*{re.escape(value_literal)}\s*;\s*$",
        flags=re.I,
    )

    if_stack: List[Dict[str, Any]] = []
    results: List[Dict[str, Any]] = []

    def _stack_repr() -> List[str]:
        rep: List[str] = []
        for e in if_stack:
            if e["branch"] == "IF":
                rep.append(f"IF@{e['if_start_line']}: {e['cond']}")
            elif e["branch"] == "ELSIF":
                rep.append(f"ELSIF@{e['if_start_line']}: {e['cond']}")
            elif e["branch"] == "ELSE":
                rep.append(f"ELSE@{e['if_start_line']} (zu IF: {e['if_cond']})")
        return rep

    for i, raw in enumerate(lines, start=1):
        line = raw.strip()
        if not line:
            continue

        m_if = rx_if.match(line)
        if m_if:
            cond = m_if.group(1).strip()
            if_stack.append({"branch": "IF", "cond": cond, "if_start_line": i, "if_cond": cond})
            continue

        m_elsif = rx_elsif.match(line)
        if m_elsif:
            cond = m_elsif.group(1).strip()
            if not if_stack:
                if_stack.append({"branch": "ELSIF", "cond": cond, "if_start_line": i, "if_cond": cond})
            else:
                top = if_stack[-1]
                top["branch"] = "ELSIF"
                top["cond"] = cond
            continue

        if rx_else.match(line):
            if if_stack:
                top = if_stack[-1]
                top["branch"] = "ELSE"
                top["cond"] = ""
            continue

        if rx_end.match(line):
            if if_stack:
                if_stack.pop()
            continue

        if rx_assign.match(line):
            snapshot = _stack_repr()
            results.append(
                {
                    "line_no": i,
                    "assignment": raw.rstrip(),
                    "conditions": snapshot,
                    "conditions_conjunction": " AND ".join(snapshot) if snapshot else "(keine IF-Bedingung im Scope)",
                }
            )

    if trace:
        trace.log(f"[ST] IF-trace for '{var_name} := {value_literal}' -> {len(results)} hits")
        for r in results:
            trace.log(f"[ST] line {r['line_no']}: {r['assignment'].strip()}")
            trace.log(f"[ST]   conditions: {r['conditions_conjunction']}")

    return results


def analyze_var_assignments_st(
    st_code: str,
    var_name: str,
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    """
    Baut eine deterministische Liste aller Zuweisungen var := TRUE/FALSE mit IF-Pfaden.
    Zus?tzlich: sortiert nach Zeilennummer und zeigt dir "last write wins"-Sicht.
    """

    true_hits = extract_set_conditions_st(st_code, var_name, "TRUE", trace=None)
    false_hits = extract_set_conditions_st(st_code, var_name, "FALSE", trace=None)

    merged: List[Dict[str, Any]] = []
    for h in true_hits:
        merged.append({**h, "value": "TRUE"})
    for h in false_hits:
        merged.append({**h, "value": "FALSE"})

    merged.sort(key=lambda x: x["line_no"])  # deterministisch nach Code-Reihenfolge

    for idx, item in enumerate(merged):
        item["has_later_assignment"] = (idx < len(merged) - 1)
        item["later_assignments"] = [{"line_no": m["line_no"], "value": m["value"]} for m in merged[idx + 1 :]]

    summary = {
        "var_name": var_name,
        "assignment_count": len(merged),
        "last_assignment_in_code": merged[-1] if merged else None,
    }

    if trace:
        trace.log(f"[ST] analyze_var_assignments_st('{var_name}') -> {len(merged)} assignments")
        for m in merged:
            trace.log(f"[ST]   line {m['line_no']}: {var_name} := {m['value']};")
            trace.log(f"[ST]     path: {m['conditions_conjunction']}")
        if merged:
            last = merged[-1]
            trace.log(f"[ST] last assignment in code: line {last['line_no']} -> {last['value']}")

    return {"summary": summary, "assignments": merged}


def find_st_true_set_lines_for_d2_path(
    ttl_path: str,
    gemma_pou_name: str = "",     # kann leer bleiben -> auto
    state_name: str = "",
    suspected_input_port: str = "",
    last_gemma_state_before_failure: str = "",
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    if trace is None:
        trace = Tracer(enabled=True, print_live=True)

    trace.log("[RUN] Starting query chain")

    g = load_graph(ttl_path, trace=trace)

    # --- GEMMA POU dynamisch ermitteln ---
    if not gemma_pou_name:
        trace.log("[RUN] gemma_pou_name not provided -> auto-detect GEMMA POU via KG flags")
        gemma_candidates = find_gemma_pous(g, trace=trace)
        if not gemma_candidates:
            trace.log("[RUN] Abort: No GEMMA CustomFBType found")
            return {"error": "Kein GEMMA CustomFBType mit dp_isGEMMAStateMachine=true gefunden.", "trace_log": trace.lines}

        pou_uri, auto_name = gemma_candidates[0]
        gemma_pou_name = auto_name
        trace.log(f"[RUN] Selected GEMMA POU deterministically: {gemma_pou_name}")
    else:
        pou_uri = find_pou_by_name(g, gemma_pou_name, trace=trace)
        if not pou_uri:
            trace.log("[RUN] Abort: GEMMA POU not found by name")
            return {"error": f"Konnte GEMMA POU '{gemma_pou_name}' nicht im KG finden.", "trace_log": trace.lines}

    fbd_code = get_pou_code(g, pou_uri, trace=trace)
    if not fbd_code:
        trace.log("[RUN] Abort: No dp_hasPOUCode for GEMMA POU")
        return {"error": f"Konnte dp_hasPOUCode f?r '{gemma_pou_name}' nicht lesen.", "trace_log": trace.lines}

    trace.log(f"[RUN] Tracing state '{state_name}' in FBD Python export")
    d2_trace = trace_d2_set_reset_from_fbd(fbd_code, state_name=state_name, trace=trace)

    if not last_gemma_state_before_failure:
        return {
            "d2_trace": d2_trace,
            "error": "last_gemma_state_before_failure ist leer. Bitte ?bergeben (z.B. 'F1').",
            "trace_log": trace.lines,
        }

    auto = infer_suspected_input_port_from_last_state(
        d2_trace,
        last_state=last_gemma_state_before_failure,
        trace=trace,
    )

    if "error" in auto:
        return {"d2_trace": d2_trace, "auto_port": auto, "error": auto["error"], "trace_log": trace.lines}

    cands = auto.get("next_trace_candidates", [])
    if not cands:
        return {"d2_trace": d2_trace, "auto_port": auto, "error": "Keine Kandidatenvariable gefunden.", "trace_log": trace.lines}

    suspected_input_port = cands[0]  # deterministisch: erster Kandidat
    trace.log(f"[RUN] Auto-detected suspected_input_port = {suspected_input_port}")

    trace.log(f"[RUN] Resolving input '{suspected_input_port}' upstream in KG")
    origin = resolve_input_to_upstream_output(g, pou_uri, suspected_input_port, trace=trace)

    if "caller_code" not in origin or not origin.get("caller_code"):
        trace.log("[RUN] Abort: No caller_code found for upstream resolution")
        return {"d2_trace": d2_trace, "origin": origin, "error": "Kein caller_code gefunden.", "trace_log": trace.lines}

    caller_port_name = origin.get("caller_port_name")
    if not caller_port_name:
        trace.log("[RUN] Abort: caller_port_name missing from origin")
        return {
            "d2_trace": d2_trace,
            "origin": origin,
            "error": "caller_port_name fehlt. Kann keine ST-Assignments suchen.",
            "trace_log": trace.lines,
        }

    st_code = origin["caller_code"]

    trace.log(f"[RUN] Extracting TRUE set lines in caller ST for variable '{caller_port_name}'")
    true_lines = extract_true_set_lines_st(st_code, caller_port_name, with_line_numbers=True, trace=trace)

    trace.log(f"[RUN] Extracting IF-conditions leading to '{caller_port_name} := TRUE;'")
    true_conditions = extract_set_conditions_st(st_code, var_name=caller_port_name, value_literal="TRUE", trace=trace)

    trace.log(f"[RUN] Extracting IF-conditions leading to '{caller_port_name} := FALSE;'")
    false_conditions = extract_set_conditions_st(st_code, var_name=caller_port_name, value_literal="FALSE", trace=trace)

    trace.log(f"[RUN] Building merged assignment order for '{caller_port_name}' (TRUE/FALSE + last-write view)")
    assignment_analysis = analyze_var_assignments_st(st_code, var_name=caller_port_name, trace=trace)

    trace.log("[RUN] Done")

    return {
        "graph": g,
        "gemma_pou_name": gemma_pou_name,
        "d2_trace": d2_trace,
        "origin": origin,
        "true_set_lines": true_lines,
        "true_set_conditions": true_conditions,
        "trace_log": trace.lines,
        "last_gemma_state_before_failure": last_gemma_state_before_failure,
        "auto_port": auto,
        "suspected_input_port": suspected_input_port,
        "false_set_conditions": false_conditions,
        "assignment_analysis": assignment_analysis,
    }


# ==========================================
# TEIL 2: DEIN ORIGINAL-CODE (Deep Trace)
# ==========================================

def _first(obj_iter): return next(iter(obj_iter), None)

def find_internal_variable_uri(graph: Graph, fbtype_uri: URIRef, var_name: str) -> Optional[URIRef]:
    for pred in (AG["op_hasInternalVariable"], AG["op_usesVariable"]):
        for v in graph.objects(fbtype_uri, pred):
            name = _first(graph.objects(v, AG["dp_hasVariableName"]))
            if name and str(name) == var_name: return v
    return None

def describe_internal_variable(graph: Graph, fbtype_uri: URIRef, var_name: str) -> Dict[str, Any]:
    var_uri = find_internal_variable_uri(graph, fbtype_uri, var_name)
    if not var_uri: return {"error": f"Interne Variable '{var_name}' nicht gefunden."}
    var_type = _first(graph.objects(var_uri, AG["dp_hasVariableType"]))
    fbinst = _first(graph.objects(var_uri, AG["op_representsFBInstance"]))
    fbinst_type = _first(graph.objects(fbinst, AG["op_isInstanceOfFBType"])) if fbinst else None
    descr = _first(graph.objects(fbinst_type, RDFS.comment)) if fbinst_type else None
    lang = _first(graph.objects(fbinst_type, AG["dp_hasPOULanguage"])) if fbinst_type else None
    return {
        "var_name": var_name, "var_uri": str(var_uri), "var_type": str(var_type) if var_type else None,
        "fbinst_uri": str(fbinst) if fbinst else None, "fbinst_type_uri": str(fbinst_type) if fbinst_type else None,
        "fbinst_type_comment": str(descr) if descr else None,
        "fbinst_type_language": str(lang) if lang else None,
    }

def find_port_uri(graph: Graph, fbtype_uri: URIRef, port_name: str) -> Optional[URIRef]:
    for p in graph.objects(fbtype_uri, AG["op_hasPort"]):
        n = _first(graph.objects(p, AG["dp_hasPortName"]))
        if n and str(n) == port_name: return p
    return None

def port_default_and_wiring(graph: Graph, fbtype_uri: URIRef, port_name: str) -> Dict[str, Any]:
    port_uri = find_port_uri(graph, fbtype_uri, port_name)
    if not port_uri: return {"note": f"'{port_name}' ist kein Port von {fbtype_uri}."}
    default_val = _first(graph.objects(port_uri, AG["dp_hasDefaultPortValue"]))
    assigns = list(graph.subjects(AG["op_assignsToPort"], port_uri))
    return {
        "port_name": port_name, "port_uri": str(port_uri), "default_value": str(default_val) if default_val else None,
        "is_wired": bool(assigns), "assignments_count": len(assigns), "assignment_uris": [str(a) for a in assigns],
    }

def get_fb_port_direction(graph: Graph, fbtype_uri: URIRef, port_name: str) -> Optional[str]:
    port_uri = find_port_uri(graph, fbtype_uri, port_name)
    if not port_uri:
        return None
    direction = _first(graph.objects(port_uri, AG["dp_hasPortDirection"]))
    return str(direction) if direction else None


def _find_symbol_roles_in_pou(graph: Graph, pou_uri: URIRef, symbol_name: str) -> Dict[str, Any]:
    """
    Best-effort \"Was ist dieses Symbol im Kontext des POU?\".

    Motivation (dein Beispiel):
      - 'Automatikbetrieb_Starten' wird im ST-Code wie eine Variable referenziert,
        ist aber im KG ein Input-Port von FB_Automatikbetrieb_F1.

    Wichtiges Kriterium:
      - Wenn op_hasPort existiert => Port-Rolle gewinnt (auch wenn es zusätzlich Var_* gibt).
    """
    port_uri = find_port_uri(graph, pou_uri, symbol_name)
    var_uri = find_internal_variable_uri(graph, pou_uri, symbol_name)

    # SPARQL-analog: <pou_uri> ?property ?searchObj . ?searchObj ?property2 "name"^^xsd:string
    # (Wir machen es ohne SPARQL, aber protokollierbar)
    hits: List[Dict[str, Any]] = []
    for prop, obj in graph.predicate_objects(pou_uri):
        if not isinstance(obj, URIRef):
            continue
        for prop2, lit in graph.predicate_objects(obj):
            if isinstance(lit, Literal) and str(lit) == symbol_name:
                hits.append({"property": str(prop), "searchObj": str(obj), "property2": str(prop2), "value": str(lit)})

    return {
        "symbol": symbol_name,
        "is_port": bool(port_uri),
        "port_uri": str(port_uri) if port_uri else None,
        "is_internal_var": bool(var_uri),
        "var_uri": str(var_uri) if var_uri else None,
        "hits": hits,
    }


def _port_assignments_to_upstream_exprs(graph: Graph, assignment_uris: List[str]) -> List[Dict[str, Any]]:
    """
    Aus op_assignsToPort-Assignments im KG einen upstream-\"Ausdruck\" erzeugen.

    Beispiel:
      Assign_MAIN_fbAuto_Automatikbetrieb_Starten_1 assignsFrom PortInstance_FBInst_MAIN_edgeF1_Q
      -> expr 'edgeF1.Q' im Kontext von Program_MAIN
    """
    out: List[Dict[str, Any]] = []
    op_assignsFrom = AG["op_assignsFrom"]
    op_instantiatesPort = AG["op_instantiatesPort"]
    op_isPortOfInstance = AG["op_isPortOfInstance"]
    dp_hasPortName = AG["dp_hasPortName"]
    dp_hasVariableName = AG["dp_hasVariableName"]
    op_representsFBInstance = AG["op_representsFBInstance"]
    op_hasAssignment = AG["op_hasAssignment"]
    op_containsPOUCall = AG["op_containsPOUCall"]

    for au in assignment_uris:
        assign_uri = URIRef(au)
        assigns_from = _first(graph.objects(assign_uri, op_assignsFrom))
        if not assigns_from:
            out.append({"assignment": au, "note": "Kein op_assignsFrom."})
            continue

        # Caller-Context via POUCall
        pou_call = _first(graph.subjects(op_hasAssignment, assign_uri))
        caller_pou = _first(graph.subjects(op_containsPOUCall, pou_call)) if pou_call else None

        # 1) PortInstance -> FBInst + PortName
        inst_port = _first(graph.objects(assigns_from, op_instantiatesPort))
        fbinst = _first(graph.objects(assigns_from, op_isPortOfInstance))
        port_name = str(_first(graph.objects(inst_port, dp_hasPortName))) if inst_port else None

        inst_var_name = None
        if fbinst:
            var = _first(graph.subjects(op_representsFBInstance, fbinst))
            inst_var_name = str(_first(graph.objects(var, dp_hasVariableName))) if var else None

        if inst_var_name and port_name:
            out.append(
                {
                    "assignment": au,
                    "caller_pou_uri": str(caller_pou) if caller_pou else None,
                    "expr": f"{inst_var_name}.{port_name}",
                    "assigns_from": str(assigns_from),
                }
            )
            continue

        # 2) Var_* (z.B. Var_MAIN_Start_eff)
        var_name_lit = _first(graph.objects(assigns_from, dp_hasVariableName))
        if var_name_lit:
            out.append(
                {
                    "assignment": au,
                    "caller_pou_uri": str(caller_pou) if caller_pou else None,
                    "expr": str(var_name_lit),
                    "assigns_from": str(assigns_from),
                }
            )
            continue

        out.append(
            {
                "assignment": au,
                "caller_pou_uri": str(caller_pou) if caller_pou else None,
                "note": "assignsFrom ist weder PortInstance noch Var_* mit dp_hasVariableName (best-effort).",
                "assigns_from": str(assigns_from),
            }
        )

    return out

def _split_top_level_commas(s: str) -> List[str]:
    parts, buf, depth, in_str = [], [], 0, False
    for i, ch in enumerate(s):
        if ch == "'" and (i == 0 or s[i-1] != "\\"): in_str = not in_str
        if not in_str:
            if ch == "(": depth += 1
            elif ch == ")": depth = max(0, depth - 1)
            elif ch == "," and depth == 0:
                parts.append("".join(buf).strip()); buf = []; continue
        buf.append(ch)
    if tail := "".join(buf).strip(): parts.append(tail)
    return parts

def _count_parens_delta(line: str) -> int:
    """
    Zählt Klammerdelta für ST-Call-Blöcke, ignoriert einfache String-Literale.
    """
    delta = 0
    in_str = False
    for i, ch in enumerate(line):
        if ch == "'" and (i == 0 or line[i - 1] != "\\"):
            in_str = not in_str
            continue
        if in_str:
            continue
        if ch == "(":
            delta += 1
        elif ch == ")":
            delta -= 1
    return delta

def find_fb_call_blocks(st_code: str, inst_name: str) -> List[Tuple[int, str]]:
    """
    Findet inst( ... ); über mehrere Zeilen.
    Rückgabe: [(start_line, full_call_block_text), ...]
    """
    clean = _strip_st_comments(st_code)
    lines = clean.splitlines()
    start_rx = re.compile(rf"^\s*{re.escape(inst_name)}\s*\(")

    blocks: List[Tuple[int, str]] = []
    collecting = False
    start_line = -1
    depth = 0
    buffer: List[str] = []

    for i, line in enumerate(lines, start=1):
        if not collecting:
            if not start_rx.match(line):
                continue
            collecting = True
            start_line = i
            depth = 0
            buffer = []

        buffer.append(line.rstrip())
        depth += _count_parens_delta(line)

        if collecting and depth <= 0 and ";" in line:
            blocks.append((start_line, "\n".join(buffer)))
            collecting = False
            start_line = -1
            depth = 0
            buffer = []

    return blocks

def parse_fb_call_args_from_block(call_block: str, inst_name: str) -> Dict[str, str]:
    m = re.search(rf"{re.escape(inst_name)}\s*\((.*?)\)\s*;", call_block, flags=re.S)
    if not m or not m.group(1).strip():
        return {}

    arg_text = m.group(1).strip()
    mapping = {}
    for a in _split_top_level_commas(arg_text):
        m_in = re.match(r"^\s*([A-Za-z_]\w*)\s*:=\s*(.+?)\s*$", a)
        m_out = re.match(r"^\s*([A-Za-z_]\w*)\s*=>\s*(.+?)\s*$", a)
        if m_in: mapping[m_in.group(1)] = m_in.group(2).strip()
        elif m_out: mapping[m_out.group(1)] = m_out.group(2).strip()
    return mapping

def find_fb_call_lines(st_code: str, inst_name: str) -> List[Tuple[int, str]]:
    # Backward compatibility für bestehende Aufrufer: Einzeiler aus Call-Block ableiten.
    out: List[Tuple[int, str]] = []
    for line_no, block in find_fb_call_blocks(st_code, inst_name):
        first = block.splitlines()[0].rstrip()
        out.append((line_no, first))
    return out

def parse_fb_call_args(call_line: str, inst_name: str) -> Dict[str, str]:
    # Backward compatibility: alter Parsername.
    return parse_fb_call_args_from_block(call_line, inst_name)

def _is_internal_v_token(var: str) -> bool:
    return bool(re.match(r"^V_\d+$", var))

def _build_fbd_true_requirement_tree(
    var: str,
    assigns: Dict[str, Tuple[str, List[str]]],
    depth: int = 0,
    max_depth: int = 25,
) -> FbdNode:
    if depth >= max_depth:
        return FbdNode(var=var, func="MAX_DEPTH", args=[])
    if var not in assigns:
        return FbdNode(var=var)

    func, args = assigns[var]
    if func.startswith("RS_") and len(args) >= 1:
        return FbdNode(
            var=var,
            func=f"{func}[TRUE->SET]",
            args=[_build_fbd_true_requirement_tree(args[0], assigns, depth + 1, max_depth)],
        )
    return FbdNode(
        var=var,
        func=func,
        args=[_build_fbd_true_requirement_tree(a, assigns, depth + 1, max_depth) for a in args],
    )

def trace_symbol_from_fbd(fbd_py_code: str, symbol_name: str, target_value: str = "TRUE") -> Dict[str, Any]:
    assigns = _parse_fbd_call_assignments(fbd_py_code, trace=None)

    root_var = None
    m = re.search(rf"^\s*{re.escape(symbol_name)}\s*=\s*(V_\d+)\s*$", fbd_py_code, flags=re.M)
    if m:
        root_var = m.group(1)
    elif symbol_name in assigns:
        root_var = symbol_name

    if not root_var:
        return {"error": f"Konnte Output-Symbol '{symbol_name}' nicht auf V_* auflösen."}

    tree = _build_fbd_tree(root_var, assigns).to_dict()
    true_tree = _build_fbd_true_requirement_tree(root_var, assigns).to_dict() if target_value.upper() == "TRUE" else tree
    return {
        "symbol": symbol_name,
        "assigned_from": root_var,
        "tree": tree,
        "target_value": target_value.upper(),
        "true_tree": true_tree,
    }

def _render_fbd_tree_lines(node: Dict[str, Any], indent: int = 0) -> List[TraceLine]:
    out: List[TraceLine] = []
    if "func" not in node:
        out.append(TraceLine(indent, f"{node.get('var')}"))
        return out
    out.append(TraceLine(indent, f"{node.get('var')} = {node.get('func')}("))
    for a in node.get("args", []) or []:
        out.extend(_render_fbd_tree_lines(a, indent + 1))
    out.append(TraceLine(indent, ")"))
    return out

def _collect_fbd_leaf_tokens(node: Dict[str, Any]) -> List[str]:
    out: List[str] = []
    if "func" not in node:
        val = str(node.get("var"))
        if val:
            out.append(val)
        return out
    for a in node.get("args", []) or []:
        out.extend(_collect_fbd_leaf_tokens(a))
    return out

def _infer_st_output_dependency_params(inst_type_name: Optional[str], output_port: str, arg_keys: List[str]) -> List[str]:
    """
    Heuristik: Wenn ein ST-Standard-FB keine expliziten Zuweisungen fuer einen Output liefert,
    leite die relevanten Input-Parameter fuer den Trace ab.
    """
    output = (output_port or "").upper()
    type_name = (inst_type_name or "").upper()

    preferred: List[str] = []
    if output == "Q":
        if type_name in ("R_TRIG", "F_TRIG", "TON", "TP", "TOF", "RS", "SR"):
            preferred = ["CLK", "IN", "S", "R", "SET", "RESET"]
        else:
            preferred = ["CLK", "IN"]

    keys_upper = {k.upper(): k for k in arg_keys}
    resolved: List[str] = []
    for p in preferred:
        if p in keys_upper and keys_upper[p] not in resolved:
            resolved.append(keys_upper[p])

    if resolved:
        return resolved
    return list(arg_keys)

def _is_global_variable_uri(graph: Graph, var_uri: URIRef) -> bool:
    scope = _first(graph.objects(var_uri, AG["dp_hasVariableScope"]))
    if scope and str(scope).strip().lower() == "global":
        return True
    if _first(graph.subjects(AG["op_listsGlobalVariable"], var_uri)):
        return True
    return False

def describe_global_variable(graph: Graph, symbol_name: str) -> Optional[Dict[str, Any]]:
    candidate_uris: List[URIRef] = []
    for v in graph.subjects(AG["dp_hasVariableName"], Literal(symbol_name)):
        if isinstance(v, URIRef) and v not in candidate_uris:
            candidate_uris.append(v)

    # Fallback: bei dotted Namen auch Kurzname pruefen (z.B. GVL.DiagnoseRequested -> DiagnoseRequested)
    if "." in symbol_name:
        short_name = symbol_name.split(".", 1)[1].strip()
        for v in graph.subjects(AG["dp_hasVariableName"], Literal(short_name)):
            if isinstance(v, URIRef) and v not in candidate_uris:
                candidate_uris.append(v)

    var_uri = None
    for v in candidate_uris:
        if _is_global_variable_uri(graph, v):
            var_uri = v
            break
    if not var_uri:
        return None

    var_type = _first(graph.objects(var_uri, AG["dp_hasVariableType"]))
    scope = _first(graph.objects(var_uri, AG["dp_hasVariableScope"]))
    bound_ports = list(graph.objects(var_uri, AG["op_isBoundToPort"]))
    bound_port_names: List[str] = []
    for p in bound_ports:
        pn = _first(graph.objects(p, AG["dp_hasPortName"]))
        if pn:
            bound_port_names.append(str(pn))

    users = list(graph.subjects(AG["op_usesVariable"], var_uri))
    user_names: List[str] = []
    for u in users:
        n = _first(graph.objects(u, AG["dp_hasPOUName"]))
        user_names.append(str(n) if n else str(u))

    list_uri = _first(graph.subjects(AG["op_listsGlobalVariable"], var_uri))
    list_name = _first(graph.objects(list_uri, AG["dp_hasGlobalVariableListName"])) if list_uri else None

    return {
        "var_uri": str(var_uri),
        "var_name": str(_first(graph.objects(var_uri, AG["dp_hasVariableName"])) or symbol_name),
        "scope": str(scope) if scope else None,
        "var_type": str(var_type) if var_type else None,
        "bound_port_names": bound_port_names,
        "used_by_pou_names": user_names,
        "global_list_uri": str(list_uri) if list_uri else None,
        "global_list_name": str(list_name) if list_name else None,
    }

def find_all_fb_call_blocks(st_code: str) -> List[Tuple[int, str, str]]:
    """
    Findet alle FB-Call-Bloecke in ST:
      [(start_line, inst_name, full_call_block), ...]
    """
    clean = _strip_st_comments(st_code)
    lines = clean.splitlines()
    start_rx = re.compile(r"^\s*([A-Za-z_]\w*)\s*\(")

    blocks: List[Tuple[int, str, str]] = []
    collecting = False
    current_inst = ""
    start_line = -1
    depth = 0
    buffer: List[str] = []

    for i, line in enumerate(lines, start=1):
        if not collecting:
            m = start_rx.match(line)
            if not m:
                continue
            collecting = True
            current_inst = m.group(1)
            start_line = i
            depth = 0
            buffer = []

        buffer.append(line.rstrip())
        depth += _count_parens_delta(line)

        if collecting and depth <= 0 and ";" in line:
            blocks.append((start_line, current_inst, "\n".join(buffer)))
            collecting = False
            current_inst = ""
            start_line = -1
            depth = 0
            buffer = []

    return blocks

def parse_fb_call_bindings_from_block(call_block: str, inst_name: str) -> List[Dict[str, str]]:
    m = re.search(rf"{re.escape(inst_name)}\s*\((.*?)\)\s*;", call_block, flags=re.S)
    if not m or not m.group(1).strip():
        return []
    arg_text = m.group(1).strip()

    out: List[Dict[str, str]] = []
    for a in _split_top_level_commas(arg_text):
        m_in = re.match(r"^\s*([A-Za-z_]\w*)\s*:=\s*(.+?)\s*$", a)
        m_out = re.match(r"^\s*([A-Za-z_]\w*)\s*=>\s*(.+?)\s*$", a)
        if m_in:
            out.append({"param": m_in.group(1), "mode": ":=", "expr": m_in.group(2).strip()})
        elif m_out:
            out.append({"param": m_out.group(1), "mode": "=>", "expr": m_out.group(2).strip()})
    return out

def find_fb_output_writers_for_variable(st_code: str, var_name: str) -> List[Dict[str, Any]]:
    hits: List[Dict[str, Any]] = []
    for line_no, inst_name, block in find_all_fb_call_blocks(st_code):
        for b in parse_fb_call_bindings_from_block(block, inst_name):
            if b["mode"] == "=>" and b["expr"].strip() == var_name:
                hits.append(
                    {
                        "line_no": line_no,
                        "inst_name": inst_name,
                        "port_name": b["param"],
                        "source_token": f"{inst_name}.{b['param']}",
                    }
                )
    return hits

def inspect_variable_access_metadata(graph: Graph, var_uri: Optional[URIRef]) -> Dict[str, Any]:
    if not var_uri:
        return {
            "has_var_uri": False,
            "dp_hasOPCUADataAccess": None,
            "dp_hasOPCUAWriteAccess": None,
            "dp_hasHardwareAddress": None,
        }

    opcua_data = _first(graph.objects(var_uri, AG["dp_hasOPCUADataAccess"]))
    opcua_write = _first(graph.objects(var_uri, AG["dp_hasOPCUAWriteAccess"]))
    hw_addr = _first(graph.objects(var_uri, AG["dp_hasHardwareAddress"]))
    return {
        "has_var_uri": True,
        "dp_hasOPCUADataAccess": str(opcua_data) if opcua_data is not None else None,
        "dp_hasOPCUAWriteAccess": str(opcua_write) if opcua_write is not None else None,
        "dp_hasHardwareAddress": str(hw_addr) if hw_addr is not None else None,
    }

def classify_symbol_in_context(graph: Graph, token: str, ctx_pou_uri: URIRef) -> Dict[str, Any]:
    """
    Zentraler Klassifikator fuer ein Token im Kontext eines POU.

    Reihenfolge:
      1) dotted token (x.y): zuerst Instanz-Port, bei Fehlschlag globale Variable
      2) plain token: Port (mit direction) > interne Variable > globale Variable
    """
    key = token.strip()

    if "." in key:
        inst, port = key.split(".", 1)
        desc = describe_internal_variable(graph, ctx_pou_uri, inst)
        if "error" not in desc:
            callee_uri = URIRef(desc["fbinst_type_uri"]) if desc.get("fbinst_type_uri") else None
            port_direction = get_fb_port_direction(graph, callee_uri, port) if callee_uri else None
            return {
                "kind": "instance_port",
                "token": key,
                "instance": inst,
                "port": port,
                "instance_desc": desc,
                "callee_uri": callee_uri,
                "callee_lang": (desc.get("fbinst_type_language") or "").upper(),
                "port_direction": port_direction,
                "reason": "dot_token->instance_port",
            }

        gvar = describe_global_variable(graph, key)
        if gvar:
            return {
                "kind": "global_var",
                "token": key,
                "global": gvar,
                "reason": "dot_token->global_var_fallback",
            }

        return {"kind": "unknown", "token": key, "reason": "dot_token->unresolved"}

    port_uri = find_port_uri(graph, ctx_pou_uri, key)
    if port_uri:
        direction = _first(graph.objects(port_uri, AG["dp_hasPortDirection"]))
        return {
            "kind": "pou_port",
            "token": key,
            "port_uri": str(port_uri),
            "port_direction": str(direction) if direction else None,
            "reason": "plain_token->pou_port",
        }

    var_uri = find_internal_variable_uri(graph, ctx_pou_uri, key)
    if var_uri:
        return {
            "kind": "internal_var",
            "token": key,
            "var_uri": str(var_uri),
            "reason": "plain_token->internal_var",
        }

    gvar = describe_global_variable(graph, key)
    if gvar:
        return {
            "kind": "global_var",
            "token": key,
            "global": gvar,
            "reason": "plain_token->global_var",
        }

    return {"kind": "unknown", "token": key, "reason": "plain_token->unresolved"}

def is_betriebsarten_layer_pou(graph: Graph, pou_uri: URIRef) -> bool:
    """
    True, wenn der aktuelle Kontext der GEMMA-Betriebsarten-Layer ist.
    """
    if not pou_uri:
        return False
    if str(pou_uri).endswith("FBType_FB_Betriebsarten"):
        return True
    name = _first(graph.objects(pou_uri, AG["dp_hasPOUName"]))
    return str(name) == "FB_Betriebsarten"

def extract_dotted_tokens(expr: str) -> List[str]:
    return re.findall(r"\b[A-Za-z_]\w*\.[A-Za-z_]\w*\b", expr)

@dataclass
class TraceLine:
    indent: int
    text: str

def trace_expr(
    graph: Graph,
    expr: str,
    fbtype_uri: URIRef,
    st_code: str,
    *,
    max_depth: int = 10,
    assumed_false_states_in_betriebsarten: Optional[Set[str]] = None,
) -> List[TraceLine]:
    lines: List[TraceLine] = []
    visited: Set[str] = set()
    assumed_false = {s.upper() for s in (assumed_false_states_in_betriebsarten or {"D1", "D2", "D3"})}

    def _last_true_assignment(var_name: str) -> Optional[Dict[str, Any]]:
        aa = analyze_var_assignments_st(st_code, var_name, trace=None)
        trues = [a for a in aa.get("assignments", []) if a.get("value") == "TRUE"]
        return trues[-1] if trues else None

    def _innermost_if_expr(conditions: List[str]) -> Optional[str]:
        rx = re.compile(r"^(IF|ELSIF)@\d+:\s*(.*)\s*$", flags=re.I)
        for raw in reversed(conditions or []):
            m = rx.match(raw.strip())
            if m:
                return m.group(2).strip()
        return None

    def rec(e: str, depth: int, ctx_pou_uri: URIRef, ctx_code: str):
        key = e.strip()
        visit_key = f"{ctx_pou_uri}|{key}"
        if visit_key in visited:
            lines.append(TraceLine(depth, f"(loop detected) {key}"))
            return
        visited.add(visit_key)

        if depth > max_depth:
            lines.append(TraceLine(depth, f"(max depth) {key}"))
            return

        if is_betriebsarten_layer_pou(graph, ctx_pou_uri) and key.upper() in assumed_false:
            lines.append(
                TraceLine(
                    depth,
                    f"[ASSUME] {key}=FALSE im GEMMA-Betriebsarten-Layer -> Zweig wird nicht weiter analysiert.",
                )
            )
            return

        cls = classify_symbol_in_context(graph, key, ctx_pou_uri)
        lines.append(TraceLine(depth, f"[CLS] token='{key}' kind={cls.get('kind')} reason={cls.get('reason')}"))

        if cls.get("kind") == "global_var":
            gvar = cls.get("global", {})
            lines.append(TraceLine(depth, f"{key} ist globale Variable (scope={gvar.get('scope')}, type={gvar.get('var_type')})."))
            if gvar.get("bound_port_names"):
                lines.append(TraceLine(depth + 1, f"op_isBoundToPort -> {gvar.get('bound_port_names')}"))
            lines.append(TraceLine(depth + 1, f"op_usesVariable -> {gvar.get('used_by_pou_names')}"))
            lines.append(TraceLine(depth + 1, "Trace endet hier (globale Signalquelle)."))
            return

        if cls.get("kind") == "instance_port":
            inst = cls["instance"]
            port = cls["port"]
            desc = cls["instance_desc"]

            lines.append(TraceLine(depth, f"{key} ist Port '{port}' der FB-Instanz '{inst}' (Typ={desc['var_type']})"))
            if desc.get("fbinst_type_comment"):
                lines.append(TraceLine(depth + 1, f"FBType-Beschreibung: {desc['fbinst_type_comment']}"))

            call_hits = find_fb_call_blocks(ctx_code, inst)
            if not call_hits:
                lines.append(TraceLine(depth + 1, f"Keine Call-Block '{inst}(...)' im ST-Code gefunden."))
                return

            call_line_no, call_block = call_hits[0]
            first_line = call_block.splitlines()[0].strip() if call_block else f"{inst}(...)"
            lines.append(TraceLine(depth + 1, f"Call-Block @ {call_line_no}: {first_line}"))
            args = parse_fb_call_args_from_block(call_block, inst)

            callee_uri = URIRef(desc["fbinst_type_uri"]) if desc.get("fbinst_type_uri") else None
            callee_lang = (desc.get("fbinst_type_language") or "").upper()
            if callee_uri:
                lines.append(TraceLine(depth + 1, f"[KG] Instanz-Typ: {callee_uri} (lang={callee_lang or 'unbekannt'})"))
            callee_port_dir = get_fb_port_direction(graph, callee_uri, port) if callee_uri else None
            lines.append(TraceLine(depth + 1, f"[KG] Port '{port}' direction im callee: {callee_port_dir}"))

            # Input-Port des callee: Argument aus Call verwenden.
            if callee_port_dir and callee_port_dir.lower() == "input":
                mapped = args.get(port)
                if mapped:
                    lines.append(TraceLine(depth + 2, f"[ST-MAP] Input-Port '{port}' <= {mapped}"))
                    for dt in extract_dotted_tokens(mapped):
                        rec(dt, depth + 3, ctx_pou_uri, ctx_code)
                    plain = mapped.strip()
                    if re.fullmatch(r"[A-Za-z_]\w*", plain):
                        rec(plain, depth + 3, ctx_pou_uri, ctx_code)
                else:
                    lines.append(TraceLine(depth + 2, f"[ST-MAP] Kein Argument für Input-Port '{port}' im Call gefunden."))
                return

            # Output-Port des callee: Sprache entscheidet den Trace-Algorithmus.
            if callee_uri and callee_lang == "FBD":
                callee_code = get_pou_code(graph, callee_uri, trace=None)
                lines.append(TraceLine(depth + 1, f"[FBD] Trace Output '{port}' im callee (FBD-Export)"))
                if not callee_code:
                    lines.append(TraceLine(depth + 2, "Kein dp_hasPOUCode im callee gefunden."))
                    return
                out_trace = trace_symbol_from_fbd(callee_code, port, target_value="TRUE")
                if "error" in out_trace:
                    lines.append(TraceLine(depth + 2, out_trace["error"]))
                    return
                lines.append(TraceLine(depth + 2, f"[FBD] {port} assigned_from {out_trace.get('assigned_from')}"))
                for tl in _render_fbd_tree_lines(out_trace["tree"], indent=depth + 3):
                    lines.append(tl)

                focus_tree = out_trace.get("true_tree") or out_trace["tree"]
                if focus_tree is not out_trace["tree"]:
                    lines.append(TraceLine(depth + 2, f"[FBD] TRUE-Pfad fuer '{port}' (RS -> SetCond):"))
                    for tl in _render_fbd_tree_lines(focus_tree, indent=depth + 3):
                        lines.append(tl)

                leaves = _collect_fbd_leaf_tokens(focus_tree)
                uniq_leaves: List[str] = []
                for tok in leaves:
                    if tok not in uniq_leaves:
                        uniq_leaves.append(tok)
                lines.append(TraceLine(depth + 2, f"[FBD] Leaf tokens (uniq): {uniq_leaves}"))

                callee_code_ctx = callee_code or ""
                for leaf in uniq_leaves:
                    leaf_dir = get_fb_port_direction(graph, callee_uri, leaf)
                    if leaf_dir and leaf_dir.lower() == "input":
                        mapped_leaf = args.get(leaf)
                        if mapped_leaf:
                            lines.append(TraceLine(depth + 2, f"[FBD->CALLMAP] {leaf} <= {mapped_leaf}"))
                            for dt in extract_dotted_tokens(mapped_leaf):
                                rec(dt, depth + 3, ctx_pou_uri, ctx_code)
                            plain = mapped_leaf.strip()
                            if re.fullmatch(r"[A-Za-z_]\w*", plain):
                                rec(plain, depth + 3, ctx_pou_uri, ctx_code)
                        else:
                            lines.append(TraceLine(depth + 2, f"[FBD->CALLMAP] Kein Call-Argument für Input '{leaf}' gefunden."))
                        continue

                    if _is_internal_v_token(leaf):
                        continue
                    if "." in leaf:
                        rec(leaf, depth + 2, callee_uri, callee_code_ctx)
                    elif re.fullmatch(r"[A-Za-z_]\w*", leaf):
                        rec(leaf, depth + 2, callee_uri, callee_code_ctx)
                    else:
                        lines.append(TraceLine(depth + 2, f"[FBD] Atom: {leaf}"))
                return

            if callee_uri and callee_lang == "ST":
                callee_code = get_pou_code(graph, callee_uri, trace=None) or ""
                lines.append(TraceLine(depth + 1, f"[ST] Trace Output '{port}' im callee (ST)"))
                aa_callee = analyze_var_assignments_st(callee_code, port, trace=None)
                summ = aa_callee.get("summary", {})
                lines.append(TraceLine(depth + 2, f"Assignments im callee ST-Code: {summ.get('assignment_count')}"))
                trues = [a for a in aa_callee.get("assignments", []) if a.get("value") == "TRUE"]
                last_true = trues[-1] if trues else None
                if last_true:
                    inn = _innermost_if_expr(last_true.get("conditions", []))
                    if inn:
                        lines.append(TraceLine(depth + 2, f"Letzte TRUE-Zuweisung @ {last_true['line_no']} (innere IF): {inn}"))
                        for tok in extract_variables_from_condition(inn):
                            rec(tok, depth + 3, callee_uri, callee_code)
                        return

                dep_params = _infer_st_output_dependency_params(desc.get("var_type"), port, list(args.keys()))
                lines.append(TraceLine(depth + 2, f"[ST-FALLBACK] Keine direkte TRUE-Zuweisung fuer Output '{port}'."))
                lines.append(TraceLine(depth + 2, f"[ST-FALLBACK] Verfolge Input-Parameter: {dep_params}"))
                for param in dep_params:
                    mapped = args.get(param)
                    if not mapped:
                        continue
                    lines.append(TraceLine(depth + 3, f"[ST-FALLBACK] {port} <= {param} <= {mapped}"))
                    for dt in extract_dotted_tokens(mapped):
                        rec(dt, depth + 4, ctx_pou_uri, ctx_code)
                    plain = mapped.strip()
                    if re.fullmatch(r"[A-Za-z_]\w*", plain):
                        rec(plain, depth + 4, ctx_pou_uri, ctx_code)
                return

            if args:
                for k_arg, v_expr in args.items():
                    lines.append(TraceLine(depth + 2, f"Param {k_arg} = {v_expr}"))
                    for dt in extract_dotted_tokens(v_expr):
                        rec(dt, depth + 3, ctx_pou_uri, ctx_code)

                    plain = v_expr.strip()
                    if re.fullmatch(r"[A-Za-z_]\w*", plain):
                        roles = _find_symbol_roles_in_pou(graph, ctx_pou_uri, plain)
                        if roles.get("hits"):
                            lines.append(TraceLine(depth + 3, f"[KG-ROLE] lookup '{plain}' -> hits={len(roles['hits'])} is_port={roles['is_port']} is_internal_var={roles['is_internal_var']}"))

                        # Port gewinnt, falls vorhanden
                        if roles.get("is_port"):
                            port_info = port_default_and_wiring(graph, ctx_pou_uri, plain)
                            dv = port_info.get("default_value")
                            wired = port_info.get("is_wired")
                            lines.append(TraceLine(depth + 3, f"'{plain}' ist Input-Port von {str(ctx_pou_uri).split('_')[-1]}"))
                            lines.append(TraceLine(depth + 4, f"default_value = {dv}"))
                            lines.append(TraceLine(depth + 4, f"is_wired = {wired} (assignments={port_info.get('assignments_count')})"))

                            if wired:
                                ups = _port_assignments_to_upstream_exprs(graph, port_info.get("assignment_uris", []))
                                for u in ups:
                                    up_expr = u.get("expr")
                                    caller_pou_uri = u.get("caller_pou_uri")
                                    lines.append(TraceLine(depth + 4, f"[KG-WIRE] {plain} <= {up_expr} (caller_pou={caller_pou_uri})"))
                                    if up_expr and caller_pou_uri:
                                        caller_uri = URIRef(caller_pou_uri)
                                        caller_code = get_pou_code(graph, caller_uri, trace=None) or ""
                                        rec(up_expr, depth + 5, caller_uri, caller_code)
                            else:
                                lines.append(TraceLine(depth + 4, "=> nicht verdrahtet, daher wird DefaultValue verwendet (Trace endet hier)."))
                        elif roles.get("is_internal_var"):
                            lines.append(TraceLine(depth + 3, f"'{plain}' ist interne Variable (wird im ST gesetzt)."))
                            aa = analyze_var_assignments_st(ctx_code, plain, trace=None)
                            summ = aa.get("summary", {})
                            lines.append(TraceLine(depth + 4, f"Assignments im ST-Code: {summ.get('assignment_count')}"))
            return

        if re.fullmatch(r"[A-Za-z_]\w*", key):
            roles = _find_symbol_roles_in_pou(graph, ctx_pou_uri, key)
            if roles.get("hits"):
                lines.append(TraceLine(depth, f"[KG-ROLE] lookup '{key}' -> hits={len(roles['hits'])} is_port={roles['is_port']} is_internal_var={roles['is_internal_var']}"))

            if cls.get("kind") == "pou_port":
                direction = (cls.get("port_direction") or "unknown").strip()
                port_info = port_default_and_wiring(graph, ctx_pou_uri, key)
                dv = port_info.get("default_value")
                wired = port_info.get("is_wired")
                lines.append(TraceLine(depth, f"'{key}' ist {direction}-Port von {str(ctx_pou_uri).split('_')[-1]}"))
                if direction.lower() == "input":
                    lines.append(TraceLine(depth + 1, f"default_value = {dv}"))
                    lines.append(TraceLine(depth + 1, f"is_wired = {wired} (assignments={port_info.get('assignments_count')})"))
                    if wired:
                        ups = _port_assignments_to_upstream_exprs(graph, port_info.get("assignment_uris", []))
                        for u in ups:
                            up_expr = u.get("expr")
                            caller_pou_uri = u.get("caller_pou_uri")
                            lines.append(TraceLine(depth + 1, f"[KG-WIRE] {key} <= {up_expr} (caller_pou={caller_pou_uri})"))
                            if up_expr and caller_pou_uri:
                                caller_uri = URIRef(caller_pou_uri)
                                caller_code = get_pou_code(graph, caller_uri, trace=None) or ""
                                rec(up_expr, depth + 2, caller_uri, caller_code)
                    else:
                        lines.append(TraceLine(depth + 1, "=> nicht verdrahtet, daher wird DefaultValue verwendet (Trace endet hier)."))
                else:
                    lines.append(TraceLine(depth + 1, "Output-Port: Wert wird intern im POU berechnet."))

                    pou_lang_lit = _first(graph.objects(ctx_pou_uri, AG["dp_hasPOULanguage"]))
                    pou_lang = str(pou_lang_lit).upper() if pou_lang_lit else ""
                    is_custom_fb = (ctx_pou_uri, RDF.type, AG["class_CustomFBType"]) in graph
                    lines.append(TraceLine(depth + 1, f"[POU-OUT] owning_pou={ctx_pou_uri} lang={pou_lang or 'unbekannt'} is_custom={is_custom_fb}"))

                    owning_code = ctx_code or (get_pou_code(graph, ctx_pou_uri, trace=None) or "")
                    if not is_custom_fb:
                        lines.append(TraceLine(depth + 1, "[POU-OUT] Kein CustomFBType. Verwende Fallback und beende hier."))
                        return
                    if not owning_code:
                        lines.append(TraceLine(depth + 1, "[POU-OUT] Kein dp_hasPOUCode gefunden."))
                        return

                    if pou_lang == "FBD":
                        lines.append(TraceLine(depth + 1, f"[POU-OUT] Rekursiver FBD-Trace fuer Output '{key}'"))
                        out_trace = trace_symbol_from_fbd(owning_code, key, target_value="TRUE")
                        if "error" in out_trace:
                            lines.append(TraceLine(depth + 2, out_trace["error"]))
                            return

                        lines.append(TraceLine(depth + 2, f"[POU-OUT][FBD] {key} assigned_from {out_trace.get('assigned_from')}"))
                        focus_tree = out_trace.get("true_tree") or out_trace.get("tree")
                        if focus_tree is not out_trace.get("tree"):
                            lines.append(TraceLine(depth + 2, f"[POU-OUT][FBD] TRUE-Pfad fuer '{key}' (RS -> SetCond):"))
                        for tl in _render_fbd_tree_lines(focus_tree, indent=depth + 3):
                            lines.append(tl)

                        leaves = _collect_fbd_leaf_tokens(focus_tree)
                        uniq_leaves: List[str] = []
                        for tok in leaves:
                            if tok not in uniq_leaves:
                                uniq_leaves.append(tok)
                        lines.append(TraceLine(depth + 2, f"[POU-OUT][FBD] Leaf tokens (uniq): {uniq_leaves}"))

                        for leaf in uniq_leaves:
                            if leaf == key or _is_internal_v_token(leaf):
                                continue
                            if "." in leaf:
                                rec(leaf, depth + 3, ctx_pou_uri, owning_code)
                            elif re.fullmatch(r"[A-Za-z_]\w*", leaf):
                                rec(leaf, depth + 3, ctx_pou_uri, owning_code)
                            else:
                                lines.append(TraceLine(depth + 3, f"[POU-OUT][FBD] Atom: {leaf}"))
                        return

                    if pou_lang == "ST":
                        lines.append(TraceLine(depth + 1, f"[POU-OUT] Rekursiver ST-Trace fuer Output '{key}'"))
                        aa_out = analyze_var_assignments_st(owning_code, key, trace=None)
                        summ_out = aa_out.get("summary", {})
                        lines.append(TraceLine(depth + 2, f"Assignments im owning ST-Code: {summ_out.get('assignment_count')}"))
                        trues_out = [a for a in aa_out.get("assignments", []) if a.get("value") == "TRUE"]
                        last_true_out = trues_out[-1] if trues_out else None
                        if last_true_out:
                            inn_out = _innermost_if_expr(last_true_out.get("conditions", []))
                            if inn_out:
                                lines.append(TraceLine(depth + 2, f"Letzte TRUE-Zuweisung @ {last_true_out['line_no']} (innere IF): {inn_out}"))
                                for tok in extract_variables_from_condition(inn_out):
                                    rec(tok, depth + 3, ctx_pou_uri, owning_code)
                        return

                    lines.append(TraceLine(depth + 1, "[POU-OUT] Unbekannte POU-Sprache. Keine weitere Rekursion."))
                    return
            elif cls.get("kind") == "internal_var":
                lines.append(TraceLine(depth, f"'{key}' ist interne Variable in {str(ctx_pou_uri).split('_')[-1]}"))
                aa = analyze_var_assignments_st(ctx_code, key, trace=None)
                summ = aa.get("summary", {})
                assign_count = int(summ.get("assignment_count") or 0)
                lines.append(TraceLine(depth + 1, f"Assignments im ST-Code: {assign_count}"))

                if assign_count == 0:
                    writer_hits = find_fb_output_writers_for_variable(ctx_code, key)
                    if writer_hits:
                        lines.append(TraceLine(depth + 1, f"[ST-FALLBACK] Output-Mappings auf '{key}': {len(writer_hits)}"))
                        for wh in writer_hits:
                            source_token = wh["source_token"]
                            lines.append(
                                TraceLine(
                                    depth + 2,
                                    f"[ST-FALLBACK] line {wh['line_no']}: {key} <= {source_token}",
                                )
                            )
                            rec(source_token, depth + 3, ctx_pou_uri, ctx_code)
                    else:
                        var_uri = URIRef(cls["var_uri"]) if cls.get("var_uri") else None
                        default_val = _first(graph.objects(var_uri, AG["dp_hasDefaultVariableValue"])) if var_uri else None
                        meta = inspect_variable_access_metadata(graph, var_uri)
                        lines.append(TraceLine(depth + 1, "[META] Keine weiteren Aufrufer gefunden. Prüfe Zugriffs-Metadaten:"))
                        lines.append(TraceLine(depth + 2, f"dp_hasOPCUADataAccess = {meta.get('dp_hasOPCUADataAccess')}"))
                        lines.append(TraceLine(depth + 2, f"dp_hasOPCUAWriteAccess = {meta.get('dp_hasOPCUAWriteAccess')}"))
                        lines.append(TraceLine(depth + 2, f"dp_hasHardwareAddress = {meta.get('dp_hasHardwareAddress')}"))
                        if default_val is not None:
                            lines.append(TraceLine(depth + 1, f"[ST-FALLBACK] Kein Writer gefunden. Default-Wert = {default_val}"))
                        else:
                            lines.append(TraceLine(depth + 1, "[ST-FALLBACK] Kein Writer und kein Default-Wert im KG gefunden."))
                    return

                last_true = _last_true_assignment(key)
                if last_true:
                    inn = _innermost_if_expr(last_true.get("conditions", []))
                    if inn:
                        lines.append(TraceLine(depth + 2, f"Letzte TRUE-Zuweisung @ {last_true['line_no']} (innere IF): {inn}"))
                        for tok in extract_variables_from_condition(inn):
                            rec(tok, depth + 3, ctx_pou_uri, ctx_code)
                else:
                    all_assignments = aa.get("assignments", [])
                    if all_assignments:
                        last_any = all_assignments[-1]
                        inn_any = _innermost_if_expr(last_any.get("conditions", []))
                        lines.append(TraceLine(depth + 2, f"Keine TRUE-Zuweisung. Letzte Zuweisung @ {last_any['line_no']} = {last_any['value']}"))
                        if inn_any:
                            lines.append(TraceLine(depth + 2, f"Innere IF der letzten Zuweisung: {inn_any}"))
                            for tok in extract_variables_from_condition(inn_any):
                                rec(tok, depth + 3, ctx_pou_uri, ctx_code)

                tail = aa.get("assignments", [])[-6:]
                if tail:
                    lines.append(TraceLine(depth + 1, "Letzte Assignments (Auszug):"))
                    for a in tail:
                        lines.append(TraceLine(depth + 2, f"@{a['line_no']} {key} := {a['value']} | Path: {a['conditions_conjunction']}"))
            else:
                lines.append(TraceLine(depth, f"'{key}' ist im Kontext nicht klassifizierbar (kind={cls.get('kind')})."))
            return

        lines.append(TraceLine(depth, f"Unbekanntes Token: {key}"))

    rec(expr, 0, fbtype_uri, st_code)
    return lines


# ==========================================
# TEIL 3: NEU - IF-BEDINGUNG ZERLEGEN & ORCHESTRIEREN
# ==========================================

def extract_variables_from_condition(cond_str: str) -> List[str]:
    """
    Extrahierte SPS-Variablen/Ports aus einem IF-Pfad (ignoriert Keywords).
    Beispiel: "IF@77: (Schritt1 OR Schritt2) AND pPer.Q" -> ['Schritt1', 'Schritt2', 'pPer.Q']
    """
    # 1. Entferne Tracer-Tags (z.B. IF@77:)
    clean_str = re.sub(r'(IF|ELSIF|ELSE)@\d+:\s*', '', cond_str)
    
    # 2. Finde alle Bezeichner (inklusive Punkte für Instanz.Port)
    tokens = re.findall(r'\b[A-Za-z_][A-Za-z0-9_]*(?:\.[A-Za-z_][A-Za-z0-9_]*)*\b', clean_str)
    
    # 3. Filtere ST-Keywords heraus
    keywords = {"AND", "OR", "NOT", "XOR", "MOD", "TRUE", "FALSE", "THEN", "ELSE", "ELSIF"}
    
    # Duplikate entfernen, Reihenfolge beibehalten
    return list(dict.fromkeys([t for t in tokens if t.upper() not in keywords]))


# ==========================================
# TEIL 3b: Bool-Analyse der IF-Bedingung
# ==========================================

@dataclass(frozen=True)
class BoolExpr:
    kind: str  # 'atom' | 'and' | 'or' | 'not'
    value: Optional[str] = None
    children: Tuple["BoolExpr", ...] = ()


def _strip_outer_parens(s: str) -> str:
    s = s.strip()
    if not (s.startswith("(") and s.endswith(")")):
        return s
    depth = 0
    in_str = False
    for i, ch in enumerate(s):
        if ch == "'" and (i == 0 or s[i - 1] != "\\"):
            in_str = not in_str
        if in_str:
            continue
        if ch == "(":
            depth += 1
        elif ch == ")":
            depth -= 1
            if depth == 0 and i != len(s) - 1:
                return s
    return s[1:-1].strip() if depth == 0 else s


def _split_top_level_bool(s: str, op: str) -> List[str]:
    op_up = op.upper()
    parts: List[str] = []
    buf: List[str] = []
    depth = 0
    in_str = False
    i = 0
    while i < len(s):
        ch = s[i]
        if ch == "'" and (i == 0 or s[i - 1] != "\\"):
            in_str = not in_str
            buf.append(ch)
            i += 1
            continue
        if not in_str:
            if ch == "(":
                depth += 1
            elif ch == ")":
                depth = max(0, depth - 1)
            if depth == 0 and s[i:].upper().startswith(op_up):
                before_ok = i == 0 or not (s[i - 1].isalnum() or s[i - 1] == "_")
                after_ok = (i + len(op_up) >= len(s)) or not (s[i + len(op_up)].isalnum() or s[i + len(op_up)] == "_")
                if before_ok and after_ok:
                    part = "".join(buf).strip()
                    if part:
                        parts.append(part)
                    buf = []
                    i += len(op_up)
                    continue
        buf.append(ch)
        i += 1
    tail = "".join(buf).strip()
    if tail:
        parts.append(tail)
    return parts


def parse_bool_expr(s: str) -> BoolExpr:
    """Sehr einfacher Parser: NOT > AND > OR; Rest wird als Atom behandelt."""
    s = _strip_outer_parens(s.strip())
    or_parts = _split_top_level_bool(s, "OR")
    if len(or_parts) > 1:
        return BoolExpr("or", children=tuple(parse_bool_expr(p) for p in or_parts))
    and_parts = _split_top_level_bool(s, "AND")
    if len(and_parts) > 1:
        return BoolExpr("and", children=tuple(parse_bool_expr(p) for p in and_parts))
    if s.upper().startswith("NOT "):
        return BoolExpr("not", children=(parse_bool_expr(s[4:].strip()),))
    return BoolExpr("atom", value=s)


def render_bool_expr(expr: BoolExpr) -> str:
    if expr.kind == "atom":
        return expr.value or ""
    if expr.kind == "not":
        return f"NOT ({render_bool_expr(expr.children[0])})"
    joiner = " AND " if expr.kind == "and" else " OR "
    return "(" + joiner.join(render_bool_expr(c) for c in expr.children) + ")"


def required_truth_paths(expr: BoolExpr, *, limit: int = 20) -> List[List[str]]:
    """Alternative Minimal-Sets an Teilbedingungen, die TRUE sein müssen.

    Beispiel: (A OR B) AND C -> [[A, C], [B, C]]
    """
    if expr.kind == "atom":
        return [[expr.value or ""]]
    if expr.kind == "not":
        inner = expr.children[0]
        if inner.kind == "atom":
            return [[f"NOT {inner.value}"]]
        return [[f"NOT ({render_bool_expr(inner)})"]]
    if expr.kind == "and":
        paths: List[List[str]] = [[]]
        for c in expr.children:
            child_paths = required_truth_paths(c, limit=limit)
            new_paths: List[List[str]] = []
            for base in paths:
                for add in child_paths:
                    new_paths.append(base + add)
                    if len(new_paths) >= limit:
                        break
                if len(new_paths) >= limit:
                    break
            paths = new_paths
        return paths[:limit]
    if expr.kind == "or":
        out: List[List[str]] = []
        for c in expr.children:
            out.extend(required_truth_paths(c, limit=limit))
            if len(out) >= limit:
                break
        return out[:limit]
    return [[render_bool_expr(expr)]]


def _extract_innermost_if_condition_from_assignment(assign: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    rx = re.compile(r"^(IF|ELSIF)@(\d+):\s*(.*)\s*$", flags=re.I)
    for raw in reversed(assign.get("conditions", []) or []):
        m = rx.match(raw.strip())
        if m:
            return {"tag": m.group(1).upper(), "if_line": int(m.group(2)), "expr": m.group(3).strip(), "raw": raw}
    return None


def run_unified_set_and_condition_trace(
    *,
    ttl_path: str,
    entrypoint: str,
    pou_name: str = "",
    target_var: str = "",
    last_gemma_state_before_failure: str = "",
    state_name: str = "D2",
    gemma_pou_name: str = "",
    max_depth: int = 12,
    max_truth_paths: int = 12,
) -> None:
    """
    Vereint Set-Trace + Deep-Trace für die *innere* (letzte) IF-Bedingung, die zu
    `target_var := TRUE;` führt.

    entrypoint:
      - 'port' / 'st': Analysiere ST-Code direkt in `pou_name` für `target_var`
      - 'fbd_state'  : nutze FBD-Export + last_gemma_state -> resolve Upstream-Port -> ST-Analyse
    """

    tr = Tracer(enabled=True, print_live=True)
    tr.log("\n=======================================================")
    tr.log("START: Unified Set-Trace + IF-Condition Deep-Trace")
    tr.log("=======================================================\n")
    tr.log(f"[CFG] ttl_path={ttl_path}")
    tr.log(f"[CFG] entrypoint={entrypoint}")

    g = load_graph(ttl_path, trace=tr)

    context_pou_uri: Optional[URIRef] = None
    context_code: Optional[str] = None
    context_target: Optional[str] = None

    if entrypoint.lower() in ("port", "st"):
        if not pou_name or not target_var:
            tr.log("[ABBRUCH] Für entrypoint='port'/'st' werden pou_name und target_var benötigt.")
            return
        pou_uri = find_pou_by_name(g, pou_name, trace=tr)
        if not pou_uri:
            tr.log(f"[ABBRUCH] POU '{pou_name}' nicht im KG gefunden.")
            return
        st_code = get_pou_code(g, pou_uri, trace=tr)
        if not st_code:
            tr.log(f"[ABBRUCH] Kein dp_hasPOUCode für '{pou_name}'.")
            return

        if entrypoint.lower() == "port":
            port_uri = get_port_by_name(g, pou_uri, target_var, trace=tr)
            if port_uri:
                direction = next(g.objects(port_uri, AG["dp_hasPortDirection"]), None)
                tr.log(f"[CTX] target_var '{target_var}' ist Port (direction={direction})")
            else:
                tr.log(f"[WARN] '{target_var}' ist als Port nicht erkennbar; fahre fort (ST-Analyse).")
        else:
            iv = find_internal_variable_uri(g, pou_uri, target_var)
            if iv:
                tr.log(f"[CTX] target_var '{target_var}' ist interne Variable (Var_* URI gefunden)")
            else:
                tr.log(f"[WARN] '{target_var}' ist als interne Variable nicht erkennbar; fahre fort.")

        context_pou_uri, context_code, context_target = pou_uri, st_code, target_var

    elif entrypoint.lower() == "fbd_state":
        if not last_gemma_state_before_failure:
            tr.log("[ABBRUCH] entrypoint='fbd_state' benötigt last_gemma_state_before_failure.")
            return
        result = find_st_true_set_lines_for_d2_path(
            ttl_path=ttl_path,
            gemma_pou_name=gemma_pou_name,
            state_name=state_name,
            suspected_input_port="",
            last_gemma_state_before_failure=last_gemma_state_before_failure,
            trace=tr,
        )
        if "error" in result:
            tr.log(f"[ABBRUCH] {result['error']}")
            return
        context_pou_uri = URIRef(result["origin"]["caller_pou_uri"])
        context_code = result["origin"]["caller_code"]
        context_target = result["origin"]["caller_port_name"]
        tr.log(f"[CTX] fbd_state resolved -> target_var='{context_target}' (caller_pou_uri={context_pou_uri})")

    else:
        tr.log(f"[ABBRUCH] Unbekannter entrypoint: {entrypoint}")
        return

    tr.log("\n--- ST: Assignment Analyse ---")
    aa = analyze_var_assignments_st(context_code, context_target, trace=tr)
    true_assignments = [a for a in aa.get("assignments", []) if a.get("value") == "TRUE"]
    dominant_assignment = true_assignments[-1] if true_assignments else None
    if not dominant_assignment:
        if entrypoint.lower() == "port":
            tr.log(f"[INFO] Keine TRUE-Zuweisung für '{context_target}' im ST-Code gefunden.")
            tr.log(f"[INFO] ENTRYPOINT='port' -> starte Deep-Trace direkt auf '{context_target}'.")
            tr.log(f"\n=== TRACE START: {context_target} ===")
            trace_lines = trace_expr(g, context_target, context_pou_uri, context_code, max_depth=max_depth)
            for tl in trace_lines:
                tr.log("  " * tl.indent + tl.text)
            tr.log(f"=== TRACE END: {context_target} ===")
            return

        tr.log(f"[ABBRUCH] Keine TRUE-Zuweisung für '{context_target}' im ST-Code gefunden.")
        return

    tr.log(f"[ST] Dominante (letzte) TRUE-Zuweisung @ {dominant_assignment['line_no']}: {dominant_assignment['assignment']}")
    tr.log(f"[ST] Path: {dominant_assignment['conditions_conjunction']}")

    inn = _extract_innermost_if_condition_from_assignment(dominant_assignment)
    if not inn:
        tr.log("[ABBRUCH] Konnte die innere IF-Bedingung nicht extrahieren.")
        return

    tr.log("\n--- Innere IF-Bedingung (Set-Trace Fokus) ---")
    tr.log(f"IF@{inn['if_line']}: {inn['expr']}")

    tr.log("\n--- Bool-Analyse (\"was musste TRUE sein?\") ---")
    bexpr = parse_bool_expr(inn["expr"])
    tr.log(f"Parsed: {render_bool_expr(bexpr)}")
    for idx, p in enumerate(required_truth_paths(bexpr, limit=max_truth_paths), start=1):
        tr.log(f"Path #{idx}: " + "  AND  ".join(p))

    tr.log("\n--- Deep Trace: Variablen/Ports aus der IF-Bedingung ---")
    vars_to_trace = extract_variables_from_condition(inn["expr"])
    tr.log(f"Extrahierte Tokens: {vars_to_trace}")

    for var in vars_to_trace:
        tr.log(f"\n=== TRACE START: {var} ===")
        trace_lines = trace_expr(g, var, context_pou_uri, context_code, max_depth=max_depth)
        for tl in trace_lines:
            tr.log("  " * tl.indent + tl.text)
        tr.log(f"=== TRACE END: {var} ===")


def run_full_automated_trace(ttl_path: str, last_gemma_state: str, target_state: str = "D2"):
    """
    Die Hauptmethode, die Skript 1 und Skript 2 vereint.
    """
    tr = Tracer(enabled=True, print_live=True)
    
    tr.log(f"\n=======================================================")
    tr.log(f"STARTE AUTOMATISCHEN GESAMT-TRACE ({target_state} nach {last_gemma_state})")
    tr.log(f"=======================================================\n")

    # 1. Methode 1 ausführen (Findet den Upstream-Port und die Zuweisungen)
    result = find_st_true_set_lines_for_d2_path(
        ttl_path=ttl_path,
        state_name=target_state,
        last_gemma_state_before_failure=last_gemma_state,
        trace=tr
    )

    if "error" in result:
        tr.log(f"\n[ABBRUCH] Es gab einen Fehler in Methode 1: {result['error']}")
        return

    # 2. Letzte "TRUE" Zuweisung ("Last write wins") finden
    assignments = result.get("assignment_analysis", {}).get("assignments", [])
    true_assignments = [a for a in assignments if a["value"] == "TRUE"]
    
    if not true_assignments:
        tr.log("\n[ABBRUCH] Es konnte keine TRUE-Zuweisung im Code gefunden werden.")
        return

    # Wir nehmen die chronologisch allerletzte TRUE Zuweisung im Code
    dominant_assignment = true_assignments[-1]
    dominant_condition = dominant_assignment["conditions_conjunction"]
    
    tr.log(f"\n--- ZERLEGUNG DER DOMINANTEN BEDINGUNG ---")
    tr.log(f"Letzte Zuweisung in Zeile {dominant_assignment['line_no']}: {result['origin']['caller_port_name']} := TRUE")
    tr.log(f"Bedingungspfad: {dominant_condition}")

    # 3. Bedingungen zerlegen
    vars_to_trace = extract_variables_from_condition(dominant_condition)
    tr.log(f"Extrahierte Trace-Variablen: {vars_to_trace}")

    # 4. Methode 2 (Deep Trace) für jede Variable aufrufen
    graph = result["graph"]
    caller_pou_uri = URIRef(result["origin"]["caller_pou_uri"])
    st_code = result["origin"]["caller_code"]

    for var in vars_to_trace:
        tr.log(f"\n=== TRACE START: {var} ===")
        trace_lines = trace_expr(graph, var, caller_pou_uri, st_code, max_depth=10)
        for tl in trace_lines:
            tr.log("  " * tl.indent + tl.text)
        tr.log(f"=== TRACE END: {var} ===")

# -----------------------------
# Ausführung
# -----------------------------
TTL_PATH = r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl"
ENTRYPOINT = "fbd_state"  # 'port' | 'st' | 'fbd_state'

# Kontext für ENTRYPOINT='port'/'st'
POU_NAME = "FB_Automatikbetrieb_F1"
TARGET_VAR = "Stoerung_erkannt"

# Kontext für ENTRYPOINT='fbd_state' (optional)
GEMMA_POU_NAME = ""  # leer -> auto GEMMA POU via KG-Flag dp_isGEMMAStateMachine
STATE_NAME = "D2"
LAST_GEMMA_STATE_BEFORE_FAILURE = "F1"

run_unified_set_and_condition_trace(
    ttl_path=TTL_PATH,
    entrypoint=ENTRYPOINT,
    pou_name=POU_NAME,
    target_var=TARGET_VAR,
    gemma_pou_name=GEMMA_POU_NAME,
    state_name=STATE_NAME,
    last_gemma_state_before_failure=LAST_GEMMA_STATE_BEFORE_FAILURE,
    max_depth=200,
    max_truth_paths=200,
)
